In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy import linalg as scpla
import scipy
# import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq,minimize
import scipy.integrate
from math import tanh,cosh
import math
import time
import matplotlib.patches as mpatches


from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

# from sympy import *
from scipy.linalg import schur, eigvals
from scipy.special import comb, perm

extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

# Help Functions 
## * connectivity structure
## * dynamics

In [ ]:
#### structural connectivity 
def generate_localstatsmat_eig_desym(Nparams,Grand,rho):
    '''
    Input
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    xaxiss,yaxiss =np.arange(NE*2),np.arange(NE*2)
    Xax,Yax=np.meshgrid(xaxiss,yaxiss)
    idxwhere = np.where(Xax>Yax)
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    X,Xsym,Xasym=np.zeros((N,N)),np.zeros((N,N)),np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    XT = X.copy().T

    Xsym = np.sqrt(rho)*X.copy()
    Xsym[idxwhere]=np.sqrt(rho)*XT[idxwhere]
    
    Xasym=np.sqrt(1-rho)*np.random.randn(NE*2,NE*2)
    X=Xsym.copy()+Xasym.copy()
    # components
    Xsym[:NE,:NE],Xsym[NE:,NE:]=xee*Xsym[:NE,:NE]/np.sqrt(NE),xii*Xsym[NE:,NE:]/np.sqrt(NI)
    Xsym[:NE,NE:],Xsym[NE:,:NE]=xei*Xsym[:NE,NE:]/np.sqrt(NE),xie*Xsym[NE:,:NE]/np.sqrt(NI)
    Xasym[:NE,:NE],Xasym[NE:,NE:]=xee*Xasym[:NE,:NE]/np.sqrt(NE),xii*Xasym[NE:,NE:]/np.sqrt(NI)
    Xasym[:NE,NE:],Xasym[NE:,:NE]=xei*Xasym[:NE,NE:]/np.sqrt(NE),xie*Xasym[NE:,:NE]/np.sqrt(NI)
    X=Xsym.copy()+Xasym.copy()
    # theoretical M (grandom) matrix
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    # for i in range(2):
    #     gmat[:,i]*=at[i]
    eigvgm,eigvecgm=la.eig(gmat)
    
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii
    return (X,Xsym,Xasym,Gamp,eigvgm,eigvecgm,gaverage)

def generate_localstatsmat_eig(Nparams,Grand):
    '''
    Input 
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nparams/N   
    X=np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,:NE],X[NE:,NE:]=xee*X[:NE,:NE]/np.sqrt(NE),xii*X[NE:,NE:]/np.sqrt(NI)
    X[:NE,NE:],X[NE:,:NE]=xei*X[:NE,NE:]/np.sqrt(NE),xie*X[NE:,:NE]/np.sqrt(NI)
    # theoretical M (grandom) matrix 
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    eigvgm,eigvecgm=la.eig(gmat)
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii

    return (X,Gamp,eigvgm,eigvecgm,gaverage)

def generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII):
    # mean value 
    # first use rank-1 structure
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nparams/N   
    Am=np.zeros((N,N))
    Am[:NE,:NE],Am[:NE,NE:]=JEE/NE,-JEI/NI
    Am[NE:,:NE],Am[NE:,NE:]=JIE/NE,-JII/NI
    Jsv=np.zeros((2,2))
    Jsv[0,0],Jsv[0,1],Jsv[1,0],Jsv[1,1]=JEE,JEI,JIE,JII

    return (Am,Jsv)

# generate gmat, same gavg
def gmatamplitude_eig(gavgfix,typenum):
    Amplit = gavgfix*typenum
    numsample = typenum**2
    Amplitg= np.zeros(numsample)
    idxc=0
    while (1):
      if idxc>=numsample:
        Amplitg[numsample-1]=1.0-np.sum(Amplitg[:numsample-1])
        break
      p=np.random.random(1)
      Amplitg[idxc]=np.minimum(p,1.0-np.sum(Amplitg))
      if np.sum(Amplitg)>1.0:
        continue 
      elif np.sum(Amplitg)==1.0:
        break
      else:
        idxc +=1
        # Amplitg[idxc]=np.min(p,1.0-np.sum(Amplitg))
    # Amplitg=0
    Amplitg*=Amplit
    Amplitg=np.sqrt(Amplitg)
    return Amplitg

def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of `x` and `y`

    Parameters
    ----------
    x, y : array_like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    Returns
    -------
    matplotlib.patches.Ellipse

    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    print(np.shape(cov))
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

def gradtanh(xorg):
    gradx = xorg.copy()
    nneuron,nknum=np.shape(gradx)[0],np.shape(gradx)[1]
    for i in range(nneuron):
        for j in range(nknum):
            gradx[i,j]=4/(np.exp(-xorg[i,j])+np.exp(xorg[i,j]))**2
    return gradx

In [ ]:
### dynamics
def iidGaussian(stats,shapem):
	mu,sig = stats[0],stats[1]
	nx,ny = shapem[0],shapem[1]
	return np.random.normal(mu,sig,(nx,ny))
def odeIntegral(x,t,J,I=0):
	x = np.squeeze(x)
	x = np.reshape(x,(len(x),1))
	# print('size:',np.shape(x),np.shape(J@np.tanh(x)))
	dxdt = -x+J@np.tanh(x)
	return np.squeeze(dxdt)
def odesimulation(t,xinit,Jpt,I):
	return scipy.integrate.odeint(partial(odeIntegral,J=Jpt,I=I),xinit,t)
## for single value 
def transferf(xorg):
	return np.tanh(xorg)
def transferdf(xorg):
    return np.log(np.cosh(xorg))
    # return 4/(np.exp(-xorg)+np.exp(xorg))**2
	# return 1/np.cosh(xorg)**2

def searchkappa_num(kappa,anmu,sigmn,xact):
	npop = len(np.squeeze(anmu))
	muphi,mudphi = np.zeros(2),np.zeros(2)
	muphi[0],muphi[1] = np.mean(transferf(xact[:NE])),np.mean(transferf(xact[NE:]))
	mudphi[0],mudphi[1] = np.mean(transferdf(xact[:NE])),np.mean(transferdf(xact[NE:]))
	rhs = 0.0
	for ip in range(npop):
		# mean
		rhs += anmu[ip] * muphi[ip] /npop
		# overlap
		rhs += sigmn[ip] * mudphi[ip]*kappa /npop
	return (kappa-rhs)

def searchkappa(kappa,anmu,sigmn,meigvec):
	npop = len(np.squeeze(anmu))
	muphi,mudphi = np.zeros(2),np.zeros(2)
	muphi[0],muphi[1] = np.mean(transferf(kappa*meigve[:NE])),np.mean(transferf(kappa*meigvec[NE:]))
	mudphi[0],mudphi[1] = np.mean(transferdf(kappa*meigvec[:NE])),np.mean(transferdf(kappa*meigvec[NE:]))
	rhs = 0.0
	for ip in range(npop):
		# mean
		rhs += anmu[ip] * muphi[ip] /npop
		# overlap
		rhs += sigmn[ip] * mudphi[ip]*kappa /npop
	return (kappa-rhs)

def decompunperturbedEI(Junperturb,nrank=1):
	eigvAm,eigveclAm,eigvecrAm=scpla.eig(Junperturb,left=True,right=True)
	meigvecAm=eigvecrAm.copy()
	neigvecAm=eigveclAm.copy()
	for i in range(nrank):
	    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
	    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
	for i in range(nrank,NE*2):
	    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
	    neigvecAm[:,i]/=normalizecoe
	return meigvecAm,neigvecAm,eigvAm
# 	
# def decompunperturbedEI(Junperturb):
# 	eigvAm,eigvecrAm=la.eig(Junperturb)
# 	meigvecAm=eigvecrAm.copy()
# 	neigvecAm=la.inv(eigvecrAm.copy())
# 	neigvecAm=neigvecAm.T
# 	for i in range(nrank):
# 	    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
# 	    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
# 	for i in range(nrank,NE*2):
# 	    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
# 	    neigvecAm[:,i]/=normalizecoe
# 	return meigvecAm,neigvecAm,eigvAm	

def decompNormalization(Jconn,refxvec,refyvec,sort=0,nrank=1):
	N = np.shape(Jconn)[0]
	NE= int(N/2)
	eigvJ,eigrvecJ=la.eig(Jconn)
	inveigrvecJ=la.inv(eigrvecJ)
	meig = np.squeeze(eigrvecJ[:,:].copy())
	neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
	neig=neig.copy().T
	for i in range(nrank):
		neig[:,i]*=eigvJ[i]
	for j in range(nrank,N):
		neig[:,j]*=eigvJ[j]
	leig, reig = np.zeros((N,N)),np.zeros((N,N))
	'''    Sort Eigenvalue in ascending     '''
	if (sort ==1):
		eigenvalue_Rsort = np.squeeze(-np.abs(np.real(eigvJ.copy())))
		idxsort=np.argsort(eigenvalue_Rsort)
		eigenvalue_sort = np.squeeze(eigvJ.copy())
		eigvJ= eigvJ[idxsort]  #### >>>>>>>>>>>>>> resorting >>>>>>>>>
		## >>> for eigendecomposition 
		reig = meig[:,idxsort]
		leig = neig[:,idxsort]
		normval=np.sum(reig*leig.copy(),axis=0)
		normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
		leig=leig.copy()/normval.copy()   # error
	elif (sort==0):	
		reig=meig.copy()
		normval=np.sum(reig*neig.copy(),axis=0)
		normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
		leig=neig.copy()/normval.copy()
	## for reference
	tildex,tildey=np.reshape(reig[:,0].copy(),(NE*2,1)),np.reshape(leig[:,0].copy(),(NE*2,1))
	## make sure the E in y is positive, for negative change signs of x and y
	# if (np.mean(tildey[:NE,0])*refyvec[0,0])<0:
	#     tildex*=(-1)
	#     tildey*=(-1)
	if (np.mean(tildex[:NE,0])*refxvec[0,0])<0:
	    tildex*=(-1)
	    tildey*=(-1)
	x,y=np.reshape(tildex,(NE*2,1)),np.reshape(tildey,(NE*2,1))
	x=np.sqrt(np.squeeze(tildey.T@refxvec)/np.squeeze(refyvec.T@tildex))*tildex
	y=np.sqrt(np.squeeze(tildex.T@refyvec)/np.squeeze(refxvec.T@tildey))*tildey
	return (eigvJ,leig,reig,x,y)

def decompUnnormalization(Jconn,refxvec,refyvec,sort=0,nrank=1):
	N = np.shape(Jconn)[0]
	NE= int(N/2)
	eigvJ,eigrvecJ=la.eig(Jconn)
	inveigrvecJ=la.inv(eigrvecJ)
	meig = np.squeeze(eigrvecJ[:,:].copy())
	neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
	neig=neig.copy().T
	for i in range(nrank):
		neig[:,i]*=eigvJ[i]
	for j in range(nrank,N):
		neig[:,j]*=eigvJ[j]
	leig, reig = np.zeros((N,N)),np.zeros((N,N))
	'''    Sort Eigenvalue in ascending     '''
	if (sort ==1):
		eigenvalue_Rsort = np.squeeze(-np.abs(np.real(eigvJ.copy())))
		idxsort=np.argsort(eigenvalue_Rsort)
		eigenvalue_sort = np.squeeze(eigvJ.copy())
		eigvJ= eigvJ[idxsort]  #### >>>>>>>>>>>>>> resorting >>>>>>>>>
		## >>> for eigendecomposition 
		reig = meig[:,idxsort]
		leig = neig[:,idxsort]
		normval=np.sum(reig*leig.copy(),axis=0)
		normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
		leig=leig.copy()/normval.copy()   # error
	elif (sort==0):	
		reig=meig.copy()
		normval=np.sum(reig*neig.copy(),axis=0)
		normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
		leig=neig.copy()/normval.copy()
	## for reference
	tildex,tildey=np.reshape(reig[:,0].copy(),(NE*2,1)),np.reshape(leig[:,0].copy(),(NE*2,1))
	## make sure the E in y is positive, for negative change signs of x and y
	# if (np.mean(tildey[:NE,0])*refyvec[0,0])<0:
	#     tildex*=(-1)
	#     tildey*=(-1)
	if (np.mean(tildex[:NE,0])*refxvec[0,0])<0:
		tildex*=(-1)
		tildey*=(-1)
	x,y=np.reshape(tildex,(NE*2,1)),np.reshape(tildey,(NE*2,1))
	return (eigvJ,leig,reig,x,y)

def Normalization(xvec,yvec,refxvec,refyvec,sort=0,nrank=1):
	N = np.shape(xvec)[0]
	NE= int(N/2)
	leig, reig = np.zeros((N,N)),np.zeros((N,N))
	reig=xvec.copy()
	normval=np.sum(reig*yvec.copy(),axis=0)
	# normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
	leig=yvec.copy()/normval.copy()
	## for reference
	tildex,tildey=np.reshape(reig[:,0].copy(),(NE*2,1)),np.reshape(leig[:,0].copy(),(NE*2,1))
	if (np.mean(tildex[:NE,0])*refxvec[0,0])<0:
		tildex*=(-1)
		tildey*=(-1)
	if (np.mean(tildex[:NE,0])*refxvec[0,0])<0:
	    tildex*=(-1)
	    tildey*=(-1)
	x,y=np.reshape(tildex,(NE*2,1)),np.reshape(tildey,(NE*2,1))
	x=np.sqrt(np.squeeze(tildey.T@refxvec)/np.squeeze(refyvec.T@tildex))*tildex
	y=np.sqrt(np.squeeze(tildex.T@refyvec)/np.squeeze(refxvec.T@tildey))*tildey
	return (leig,reig,x,y)

def numerical_stats(xrvec,ylvec,xrref,ylref,eigv,nrank,npop,ppercent):
	nneuron = np.shape(xrvec)[0]
	nnpop = np.zeros(npop)
	for i in range(npop):
		nnpop[i] = int(ppercent[i]*nneuron)
	axrmu,aylmu = np.zeros((npop,nrank)),np.zeros((npop,nrank))
	sigxr,sigyl = np.zeros((npop,nrank)),np.zeros((npop,nrank))
	sigcov = np.zeros((npop,nrank,nrank))

	for irank in range(nrank):
		for ipop in range(npop):
			nns,nne = np.sum(nnpop[:ipop]),np.sum(nnpop[:ipop])+nnpop[ipop]
			nns = nns.astype(np.int32)
			nne = nne.astype(np.int32)
			axrmu[ipop,irank],aylmu[ipop,irank] = np.mean(xrvec[nns:nne,irank]),np.mean(ylvec[nns:nne,irank])
			sigxr[ipop,irank],sigyl[ipop,irank] = np.std(xrvec[nns:nne,irank])**2,np.std(ylvec[nns:nne,irank])**2

			# axrmu[ipop,irank],aylmu[ipop,irank] = np.mean(xrref[nns:nne,irank]),np.mean(ylref[nns:nne,irank])
			# sigxr[ipop,irank],sigyl[ipop,irank] = np.mean((xrvec[nns:nne,irank]-xrref[nns:nne,irank])**2),np.mean((ylvec[nns:nne,irank]-ylref[nns:nne,irank])**2)

	for ipop in range(npop):
		neuronpop = int(nnpop[ipop])
		nns,nne = np.sum(nnpop[:ipop]),np.sum(nnpop[:ipop])+nnpop[ipop]
		nns = nns.astype(np.int32)
		nne = nne.astype(np.int32)
		noisexr,noiseyl = np.zeros((neuronpop,nrank)),np.zeros((neuronpop,nrank))
		for irank in range(nrank):
			noisexr[:,irank],noiseyl[:,irank] = xrvec[nns:nne,irank],ylvec[nns:nne,irank]
			noisexr[:,irank]-=axrmu[irank]
			noiseyl[:,irank]-=aylmu[irank]
		sigcov[ipop,:,:] = noiseyl.T@noisexr/neuronpop

	return(axrmu,aylmu,sigxr,sigyl,sigcov)

In [ ]:
### Statistics of Gaussian random matrix 
### Gaussian Pparameters
gaussian_norm = (1/np.sqrt(np.pi))
gauss_points, gauss_weights = np.polynomial.hermite.hermgauss(300)
gauss_points = gauss_points*np.sqrt(2)

def Phi(mu, delta0):
    integrand = np.tanh(mu+np.sqrt(delta0)*gauss_points)
    return gaussian_norm * np.dot (integrand,gauss_weights)

def derPhi(mu,delta0):
    # integrand = np.log(np.cosh(mu+np.sqrt(delta0)*gauss_points))
    integrand = 2/(np.cosh(2*(mu+np.sqrt(delta0)*gauss_points))+1.0)
    return gaussian_norm * np.dot (integrand,gauss_weights)

def innerdeuxPhi(mu,delta0):
    # integrand = np.tanh(mu+np.sqrt(delta0)*gauss_points)
    # return gaussian_norm * np.dot(integrand,gauss_weights)
    integrand = np.tanh(mu+np.sqrt(delta0)*gauss_points)
    return gaussian_norm * np.dot (integrand**2,gauss_weights)

def innerdeuxderPhi(mu,delta0):
    # integrand = np.log(np.cosh(mu+np.sqrt(delta0)*gauss_points))
    integrand = 2/(np.cosh(2*(mu+np.sqrt(delta0)*gauss_points))+1.0)
    return gaussian_norm * np.dot (integrand**2,gauss_weights)

### no correlation
## solve the dynamics using the consistency of kappa
def iidperturbation(x,JE,JI,g0):
    # JE,JI,g0 = args
    muphi,delta0phi = x, x**2*g0**2/(JE-JI)**2
    delta_kappa = -x+(JE-JI)*Phi(muphi,delta0phi)
    return delta_kappa

## solve the dynamics using the consistency of mu and sigma    
def iidfull_mudelta_consistency(x,JE,JI,g0):
    mux,sigx2 = x[0],x[1]
    inner2mean = innerdeuxPhi(mux,sigx2)
    Phimean = Phi(mux,sigx2)
    consistency = [x[1]-g0**2*inner2mean,x[0]-(JE-JI)*Phimean]
    return consistency
def iidR1_mudelta_consistency(x,JE,JI,g0):
    mux,sigx2 = x[0],x[1]
    outer2mean = Phi(mux,sigx2)**2
    Phimean = Phi(mux,sigx2)
    consistency = [x[1]-g0**2*outer2mean,x[0]-(JE-JI)*Phimean]
    return consistency

## solve the dynamics using the consistency of kappa (sym)
def symperturbation(x,JE,JI,g0,etaset):
    # JE,JI,g0 = args
    muphi,delta0phi = x, x**2*g0**2/(JE-JI)**2
    delta_kappa = -x+(JE-JI)*Phi(muphi,delta0phi)
    sigmaE_term = g0**2*(JE*etaset[0]-JI*etaset[1])/(JE-JI)**2*derPhi(muphi,delta0phi)*x/2.0
    sigmaI_term = g0**2*(JE*etaset[1]-JI*etaset[5])/(JE-JI)**2*derPhi(muphi,delta0phi)*x/2.0
    delta_kappa = delta_kappa+(sigmaE_term+sigmaI_term)
    return delta_kappa
## solve the dynamics using the consistency of mu and sigma (sym)
def symfull_mudelta_consistency(x,JE,JI,g0):
    mux,sigx2 = x[0],x[1]
    inner2mean = innerdeuxPhi(mux,sigx2)
    Phimean = Phi(mux,sigx2)
    consistency = [x[1]-g0**2*inner2mean,x[0]-(JE-JI)*Phimean]
    return consistency

def symR1_mudelta_consistency(x,JE,JI,g0):
    mux,sigx2 = x[0],x[1]
    outer2mean = Phi(mux,sigx2)**2
    Phimean = Phi(mux,sigx2)
    consistency = [x[1]-g0**2*outer2mean,x[0]-(JE-JI)*Phimean]
    return consistency


### functions compare/how variance and mean co-change under the condition with iid Random Perturbation
def iid_muCdelta_consistency(x,JE,JI,g0, sigx2):
    mux = x[0]
    Phimean = Phi(mux,sigx2)
    consistency = x[0]-(JE-JI)*Phimean
    return consistency



## with i.i.d Random Connectivity -- Rank-one Unperturbed Matrix
## * statistics change with $g_0$;
## * statistics change with $J_E$

In [ ]:
'''
WITHOUT SYMMETRY, 
* increase g0 from zero to considerable
* constant JE,JI,a,b
'''
from functools import partial
# generate mean matrix
nrank=1
Nt=np.array([300,300])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
nrank,ntrial,neta,nvec,nmaxFP=1,60,1,2,3
''' Network Setting  '''
## heterogeneous random gain
ngavg = 9
gavgseries = np.linspace(0.0,0.8,ngavg)
xee,xei,xie,xii=1.0,1.0,1.0,1.0 

## heterogeneous degree of symmetry: amplitudes and signs
coeffetaEsub=np.array([1.0,1.0,1.0])#
coeffetaTotal = np.array([1.0,1.0,1.0])#
# coeffetaTotal = np.zeros(3)
signetaEsub=np.ones(3)
signetaTotal=np.ones(3)

ppercentEsub = np.ones(2)
ppercentEsub[0]=0.5
ppercentEsub[1]=1.0-ppercentEsub[0]
## E->total ppercentEsub[0]/2.0,ppercentEsub[1]/2.0, (I) 1/2.0

'''  Recording Variables. '''
## perturbed + symmetric eigenvectors
Reigvecseries,Leigvecseries=np.zeros((ngavg,ntrial,NE*2,nvec*2)),np.zeros((ngavg,ntrial,NE*2,nvec*2))
ReigvecTseries,LeigvecTseries=np.zeros((ngavg,ntrial,NE*2,nvec*2)),np.zeros((ngavg,ntrial,NE*2,nvec*2))
Beigvseries=np.zeros((ngavg,ntrial,N),dtype=complex)
## corresponding statistics \mu and \sigma \cov
armu,sigrcov = np.zeros((ngavg,ntrial,2)),np.zeros((ngavg,ntrial,2)) # 2 for E and I
almu,siglcov = np.zeros((ngavg,ntrial,2)),np.zeros((ngavg,ntrial,2))
siglr = np.zeros((ngavg,ntrial,2))
## reference, eigenvectors of matrix with iid perturbations
x0series,y0series=np.zeros((ngavg,ntrial,NE*2)),np.zeros((ngavg,ntrial,NE*2))
gYbasexee=np.zeros((ntrial,NE*2,NE*2))

''' Nullclines parameters  '''
kappaintersect_Full = np.zeros((ngavg,ntrial,nmaxFP*2))
kappaintersect_R1   = np.zeros((ngavg,ntrial,nmaxFP*2))

''' simulating parameters '''
tt = np.linspace(0,150,500)
dt = tt[2]-tt[1]
ntt = len(tt)
xfpseries_Full = np.zeros((ngavg,ntrial,NE*2,ntt))
xfpseries_R1   = np.zeros((ngavg,ntrial,NE*2,ntt))
''' All random matrices for each trials '''
''' ## Three \bar{J} cases '''
JI,JE,a,b=0.5,1.8,0,0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
''' Am -- J(g0=0), xAm(g0=0), yAm(g0=0) '''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
''' for reference, eigenvectors of unperturbed matrix \bar{J}'''
meigvecAm,neigvecAm,eigvAm = decompunperturbedEI(Am)
xAm,yAm=np.reshape(meigvecAm[:,0],(NE*2,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(NE*2,1))

''' Iterative Processing '''
for iktrial in range(ntrial):
    print('>>>>>>> simulating neuronal activity ... >>>>>>')
    '''    ##>>>>>>>>>>> g0!=0, >>>>>>>>     '''
    Xsym =iidGaussian([0,1.0/np.sqrt(NE*2)],[NE*2,NE*2])
    XsymT=Xsym.copy().T
    X0=Xsym.copy()
    ## log: there is a bug, X is the total random xconn, including symmetry
    gYbasexee[iktrial,:,:]=Xsym.copy()
    for idxg,gaverage in enumerate(gavgseries):
        eta = 0
        etaset=eta*np.ones(6)
        for iloop in range(3):
            etaset[iloop] = etaset[iloop]*coeffetaEsub[iloop]
            etaset[iloop+3] = etaset[iloop+3]*coeffetaTotal[iloop]
        Xinit = Xsym.copy()            
        X=Xinit.copy()
        X[:NE,:NE]*=(xee*gaverage)
        X[:NE,NE:]*=(xei*gaverage)
        X[NE:,:NE]*=(xie*gaverage)
        X[NE:,NE:]*=(xii*gaverage) # adding gaverage here, check the random gain by the radius of bulk

        # overall
        J = X.copy()+Am.copy()
        JT = J.copy().T
        ''' iid Gaussian randomness '''
        eigvJ,leigvec,reigvec,xnorm0,ynorm0=decompNormalization(J,xAm,yAm,sort=0)
        ''' ## P = mn.T/N, already have rl.T -- r(m) \lambda l.T(n.T) -- Reig*\sqrt(N)Leig*\sqrt(N)/N   '''
        xnorm0,ynorm0 = xnorm0*np.sqrt(2*NE),ynorm0*np.sqrt(2*NE)
        Reigvecseries[idxg,iktrial,:,0],Leigvecseries[idxg,iktrial,:,0]=xnorm0[:,0].copy(),ynorm0[:,0].copy()
        Beigvseries[idxg,iktrial,:]=eigvJ.copy()

        axrmu,aylmu,sigxr,sigyl,sigcov = numerical_stats(xnorm0,ynorm0,xAm*np.sqrt(NE*2),yAm*np.sqrt(NE*2),eigvJ,nrank,2,ppercent=np.array([0.5,0.5]))
        armu[idxg,iktrial,:],almu[idxg,iktrial,:] = axrmu[:,0],aylmu[:,0]
        sigrcov[idxg,iktrial,:],siglcov[idxg,iktrial,:]= sigxr[:,0],sigyl[:,0]
        siglr[idxg,iktrial,:] = sigcov[:,0,0]
        
        ## Rank One Approximation, Nullclines of \kappa
        xnorm0, ynorm0 = Reigvecseries[idxg,iktrial,:,0].copy(),Leigvecseries[idxg,iktrial,:,0]
        xnorm0, ynorm0 = np.reshape(xnorm0,(2*NE,1)),np.reshape(ynorm0,(2*NE,1))
        xmu,ymu = armu[idxg,iktrial,:].copy(),almu[idxg,iktrial,:].copy()
        xsig,ysig = sigrcov[idxg,iktrial,:].copy(),siglcov[idxg,iktrial,:].copy()
        yxcov = siglr[idxg,iktrial,:].copy()

        ''' Full Connectivity -- Dynamics '''
        Jpt = J.copy()
        xinit = np.random.normal(0,1E-2,(1,NE*2))
        xinit = np.squeeze(xinit)
        xtemporal = odesimulation(tt,xinit,Jpt,0)
        xfpseries_Full[idxg,iktrial,:,:] = xtemporal.T.copy()
        ## 2 KAPPA_0
        kappanum = np.zeros(3)
        xact = np.squeeze(xfpseries_Full[idxg,iktrial,:,-1])
        ## use yAm -- unperturbed centre.
        if((ymu[0]*eigvJ[0])*(yAm[0,0]*eigvAm[0])>0):
            kappanum[0] = yAm[0,0]*np.mean(np.tanh(xact[:NE]))/2.0+yAm[NE,0]*np.mean(np.tanh(xact[NE:]))/2.0
        else:
            kappanum[0] = -yAm[0,0]*np.mean(np.tanh(xact[:NE]))/2.0-yAm[NE,0]*np.mean(np.tanh(xact[NE:]))/2.0
        kappanum[0]=kappanum[0]*np.sqrt(NE*2)*eigvAm[0]
        kappaintersect_Full[idxg,iktrial,:3]=kappanum[:].copy()

        ''' use perturbation theorem to calculate the perturbed vectors ''' 
        xnormt, ynormt = X.copy()@xAm.copy()/eigvAm[0], X.copy().T@yAm.copy()/eigvAm[0]#yAm.copy().T@X/eigvAm[0]*np.sqrt(N)
        xnormt, ynormt = xnormt+xAm.copy(), ynormt+yAm.copy()

        ## normalize 
        xnormt = xnormt.copy()/np.linalg.norm(xnormt.copy())
        ynormt = ynormt.copy()
        _,_,xnormt,ynormt=Normalization(xnormt.copy(),ynormt.copy(),xAm.copy(),yAm.copy(),sort=0,nrank=1)
        ReigvecTseries[idxg,iktrial,:,0],LeigvecTseries[idxg,iktrial,:,0]=xnormt[:,0].copy(),ynormt[:,0].copy()
        xnormt,ynormt = xnormt*np.sqrt(NE*2),ynormt*np.sqrt(NE*2)
        # xnormt, ynormt = X.copy()@xAm.copy()/eigvAm[0]*np.sqrt(NE*2), X.copy().T@yAm.copy()/eigvAm[0]*np.sqrt(NE*2)#yAm.copy().T@X/eigvAm[0]*np.sqrt(N)
        # xnormt, ynormt = xnormt+xAm.copy()*np.sqrt(NE*2), ynormt+yAm.copy()*np.sqrt(NE*2)
        # ReigvecTseries[idxg,iktrial,:,0],LeigvecTseries[idxg,iktrial,:,0]=xnormt[:,0].copy(),ynormt[:,0].copy()
        r1Matt = np.real(xnormt@ynormt.T)
        r1Matt = r1Matt*np.real(eigvAm[0])/NE/2.0
        ### START SIMULATING AND CALCULATE KAPPA AND xfpsereis_R1
        ## use the same initial values
        xtemporal = odesimulation(tt,xinit,r1Matt,0)
        xfpseries_R1[idxg,iktrial,:,:] = xtemporal.T.copy()
        ''' kappa dynamics    '''
        ## 2 populations 
        kappanum = np.zeros(3)
        xact = np.squeeze(xfpseries_R1[idxg,iktrial,:,-1])
        ## use yAm -- unperturbed centre.
        if((ymu[0]*eigvJ[0])*(yAm[0,0]*eigvAm[0])>0):
            # do not change
            kappanum[0] = yAm[0,0]*np.mean(np.tanh(xact[:NE]))/2.0+yAm[NE,0]*np.mean(np.tanh(xact[NE:]))/2.0
        else:
            kappanum[0] = -yAm[0,0]*np.mean(np.tanh(xact[:NE]))/2.0-yAm[NE,0]*np.mean(np.tanh(xact[NE:]))/2.0
        kappanum[0]=kappanum[0]*np.sqrt(NE*2)*eigvAm[0]
        kappaintersect_R1[idxg,iktrial,:3]=kappanum[:].copy()


In [ ]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE
Randgain=np.zeros(4)
Randgain[0],Randgain[1],Randgain[2],Randgain[3]=xee,xei,xie,xii

np.savez(now+'_iidDynamics_g0_data',Jparameters=Jparameters,JEseries=JEseries,Randgain=Randgain,gaverage=gaverage,Reigvecseries=Reigvecseries[:,:,:,0],Leigvecseries=Leigvecseries[:,:,:,0],ReigvecTseries=ReigvecTseries[:,:,:,0],LeigvecTseries=LeigvecTseries[:,:,:,0],Beigvseries=Beigvseries[:,:,0],gYbasexee=gYbasexee,kappaintersect_Full=kappaintersect_Full,kappaintersect_R1=kappaintersect_R1,xfpseries_Full=xfpseries_Full[:,:,:,-1],xfpseries_R1=xfpseries_R1[:,:,:,-1])

## theoretical values of $\kappa$, using the self-consistency in the first moment $\mu$
## considering $\mu=\kappa\mu_m$, and $\Delta=\kappa^2\sigma_{m^2}$
* self-consistency in $\kappa$ only for rank one approximated network, as to the full connectivity, consider the self-consistency in $\mu$ and $\Delta$

In [ ]:
## self-consistency in \kappa, for rank one approximation 
kappa_theo_iid = np.zeros((ngavg,3))
for idxg, gaverage in enumerate(gavgseries):
    init_k=1.5
    kappa_theo_iid[idxg,0]= fsolve(iidperturbation,init_k,args=(JE,JI,gaverage))
    init_k=0.0
    kappa_theo_iid[idxg,1]= fsolve(iidperturbation,init_k,args=(JE,JI,gaverage))
    init_k=-1.5
    kappa_theo_iid[idxg,2]= fsolve(iidperturbation,init_k,args=(JE,JI,gaverage))
## plot kappa change with JE ##
fig,ax = plt.subplots(figsize=(4,4))
ax.plot(gavgseries,kappa_theo_iid[:,0],color='black',linewidth=1.5)
ax.plot(gavgseries,kappa_theo_iid[:,1],color='gray',linewidth=1.5)
ax.plot(gavgseries,kappa_theo_iid[:,2],color='black',linewidth=1.5)
for iktrial in range(ntrial):
    ax.plot(gavgseries,kappaintersect_Full[:,iktrial,0],'^',color='blue',markersize=5)
    ax.plot(gavgseries,kappaintersect_R1[:,iktrial,0],'^',color='green',markersize=5)

## compare the statistics of population activity
* for full connectivity
    * numerical mean and variancec mu_x_num, variance_x_num
    * theorectical (self-consistency) mu_full_theo, variance_full_theo
* for rank one approximation.
    * numerical mean and variance mu_kappa_num, variance_kappa_num
    * theoretical (self-consistency, for $mu_{R1}$ and $Delta_{R1}$ and also the self-consistency in $\kappa_{R1}$)
* compare the previous work
    * $\Delta_{iid}=g_0^2\langle\phi^2\rangle$, $\mu_{iid}=\bar{\mathbf{n}}^{\intercal}\phi/N$

In [ ]:
### test two expressions of variance ##
## numerically calculate the variance and mean, using xfpseries_Full and xfpseries_R1
variance_x_num,variance_kappa_num = np.zeros((ngavg,ntrial,2)),np.zeros((ngavg,ntrial,2))
mu_x_num,mu_kappa_num = np.zeros((ngavg,ntrial,2)),np.zeros((ngavg,ntrial,2))
variance_gavg_num, mu_gavg_num =np.zeros((ngavg,ntrial,2)), np.zeros((ngavg,ntrial,2))
## analytically calculate the variance and mean, using the expression (self-consistency)
variance_full_theo, mu_full_theo = np.zeros((ngavg,ntrial,2)),np.zeros((ngavg,ntrial,2))
variance_R1_theo, mu_R1_theo = np.zeros((ngavg,ntrial,2)),np.zeros((ngavg,ntrial,2))
for idxg, gaverage in enumerate(gavgseries):
    for iktrial in range(ntrial):
        ## numerical results for Full Mat
        variance_x_num[idxg,iktrial,0],variance_x_num[idxg,iktrial,1]=np.std(xfpseries_Full[idxg,iktrial,:NE,-1])**2,np.std(xfpseries_Full[idxg,iktrial,NE:,-1])**2
        mu_x_num[idxg,iktrial,0],mu_x_num[idxg,iktrial,1]=np.mean(xfpseries_Full[idxg,iktrial,:NE,-1]),np.mean(xfpseries_Full[idxg,iktrial,NE:,-1])
        ## numerical results for Rank one Appriximation Mat
        variance_kappa_num[idxg,iktrial,0],variance_kappa_num[idxg,iktrial,1]=np.std(xfpseries_R1[idxg,iktrial,:NE,-1])**2,np.std(xfpseries_R1[idxg,iktrial,NE:,-1])**2
        mu_kappa_num[idxg,iktrial,0],mu_kappa_num[idxg,iktrial,1]=np.mean(xfpseries_R1[idxg,iktrial,:NE,-1]),np.mean(xfpseries_R1[idxg,iktrial,NE:,-1])
        # ## gavg inner-square
        xe,xi = np.squeeze(xfpseries_Full[idxg,iktrial,:NE,-1]),np.squeeze(xfpseries_Full[idxg,iktrial,NE:,-1])
        variance_gavg_num[idxg,iktrial,0],variance_gavg_num[idxg,iktrial,1]  = gaverage**2*np.mean(np.tanh(xe)**2),gaverage**2*np.mean(np.tanh(xi)**2) # still use numerical neuronal activities -- realizations
        nvec = np.reshape(np.squeeze(Leigvecseries[idxg,iktrial,:,0]),(2*NE,1))*Beigvseries[idxg,iktrial,0]
        kappa_classic = np.squeeze(nvec.T@np.reshape(np.squeeze(xfpseries_Full[idxg,iktrial,:,-1]),(NE*2,1))/NE/2.0)
        mu_mvecE,mu_mvecI = np.mean(np.squeeze(Reigvecseries[idxg,iktrial,:NE,0])),np.mean(np.squeeze(Reigvecseries[idxg,iktrial,NE:,0]))
        mu_gavg_num[idxg,iktrial,0],mu_gavg_num[idxg,iktrial,1] = mu_mvecE*kappa_classic,mu_mvecI*kappa_classic
        
for idxg, gaverage in enumerate(gavgseries):
    # ## theoretical    
    deltainit,meaninit = np.mean(variance_x_num[idxg,:,0]),kappa_theo_iid[idxg,0]#np.mean(kappaintersect[idxg,:,0])
    statsfull = fsolve(iidfull_mudelta_consistency,[meaninit,deltainit],args=(JE,JI,gaverage))
    mu_full_theo[idxg,:,0],variance_full_theo[idxg,:,0]= statsfull[0],statsfull[1]*np.ones(ntrial)
    mu_full_theo[idxg,:,1],variance_full_theo[idxg,:,1]= statsfull[0],statsfull[1]*np.ones(ntrial)
    ## use mean and variance consistency
    statsfull = fsolve(iidR1_mudelta_consistency,[meaninit,deltainit],args=(JE,JI,gaverage))
    mu_R1_theo[idxg,:,0],variance_R1_theo[idxg,:,0]= statsfull[0]*np.ones(ntrial),statsfull[1]*np.ones(ntrial)
    mu_R1_theo[idxg,:,1],variance_R1_theo[idxg,:,1]= statsfull[0]*np.ones(ntrial),statsfull[1]*np.ones(ntrial)
    ## use kappa consistency
    # mu_R1_theo[idxg,:,0],variance_R1_theo[idxg,:,0]= kappa_theo_iid[idxg,0],kappa_theo_iid[idxg,0]**2*gaverage**2/(JE-JI)**2
    # mu_R1_theo[idxg,:,1],variance_R1_theo[idxg,:,1]= kappa_theo_iid[idxg,0],kappa_theo_iid[idxg,0]**2*gaverage**2/(JE-JI)**2

import matplotlib.patches as mpatches
### WAY ONE --- for variance
xticks = np.linspace(.0,0.30,2)
xlims = [-.05,0.35]
yticks = np.linspace(.0,0.30,2)
ylims = [-.05,0.35]

vmeanxnum,vmeankappanum = np.zeros((ngavg,2)),np.zeros((ngavg,2))
mmeanxnum,mmeankappanum = np.zeros((ngavg,2)),np.zeros((ngavg,2))
for i in range(2):
        vmeanxnum[:,i] = np.mean(variance_x_num[:,:,i],axis=1)
        vmeankappanum[:,i] = np.mean(variance_kappa_num[:,:,i],axis=1)
        mmeanxnum[:,i] = np.mean(mu_x_num[:,:,i],axis=1)
        mmeankappanum[:,i] = np.mean(mu_kappa_num[:,:,i],axis=1)

fig,ax2 = plt.subplots(1,2,figsize=(6,4))
positions = np.arange(0, 5*ngavg-1, 5)
for iktrial in range(ntrial):
        ax2[0].plot(variance_x_num[:,iktrial,0],variance_kappa_num[:,iktrial,0],'^',markersize=1,color='red',label=r'$\Delta^E$')
        ax2[1].plot(variance_x_num[:,iktrial,1],variance_kappa_num[:,iktrial,1],'^',markersize=1,color='blue',label=r'$\Delta^I')
        ax2[0].plot(variance_full_theo[:,iktrial,0],variance_R1_theo[:,iktrial,0],'o',markersize=5,color='orange')#'black')
        ax2[1].plot(variance_full_theo[:,iktrial,0],variance_R1_theo[:,iktrial,0],'o',markersize=5,color='green')#'black')

## auxiliary 
xxx = np.linspace(0.0,0.50,2)
for i in range(2):
        # varnum = np.polyfit(np.squeeze(variance_x_num[:3,:,i]).flatten(), np.squeeze(variance_kappa_num[:3,:,i]).flatten(), deg=1)
        # yvar=varnum[0]*xxx+varnum[1]
        # ax2[i].plot(xxx,yvar,linestyle='-',color='gray')   
        ax2[i].plot(xxx,xxx,linestyle='--',color='black')             
        ax2[i].set_xlim(xlims)
        ax2[i].set_ylim(ylims)
        ax2[i].set_xticks(xticks)
        ax2[i].set_yticks(yticks)
        ax2[i].set_aspect('equal')
xticks = np.linspace(.5,1.0,2)
xlims = [0.4,1.1]
yticks = np.linspace(.5,1.0,2)
ylims = [0.4,1.1]

fig,ax2 = plt.subplots(1,2,figsize=(6,4))
positions = np.arange(0, 5*ngavg-1, 5)
medianxtheo,mediankappatheo = np.zeros((ngavg,2)),np.zeros((ngavg,2))
for iktrial in range(ntrial):
        ax2[0].plot(np.abs(mu_x_num[:,iktrial,0]),np.abs(mu_kappa_num[:,iktrial,0]),'^',markersize=1,color='red',label=r'$\mu^E$')
        ax2[1].plot(np.abs(mu_x_num[:,iktrial,1]),np.abs(mu_kappa_num[:,iktrial,1]),'^',markersize=1,color='blue',label=r'$\mu^I$')
        ax2[0].plot(mu_full_theo[:,iktrial,0],mu_R1_theo[:,iktrial,0],'o',markersize=5,color='orange')#'black')
        ax2[1].plot(mu_full_theo[:,iktrial,0],mu_R1_theo[:,iktrial,0],'o',markersize=5,color='green')#'black')

xxx = np.linspace(0.0,1.50,2)
for i in range(2):
        ax2[i].plot(xxx,xxx,linestyle='--',color='black') 
        # munum = np.polyfit(np.squeeze(mu_x_num[:,:,i]).T.flatten(), np.squeeze(mu_kappa_num[:,:,i]).T.flatten(), deg=1)
        # ymu=munum[0]*xxx+munum[1]
        # ax2[i].plot(xxx,ymu,linestyle='-',color='gray')      
        ax2[i].set_xlim(xlims)
        ax2[i].set_ylim(ylims)
        ax2[i].set_xticks(xticks)
        ax2[i].set_yticks(yticks)
        ax2[i].set_aspect('equal')


In [ ]:
def add_label(violin, label):
    color = violin["bodies"][0].get_facecolor().flatten()
    labels.append((mpatches.Patch(color=color), label))
labels = []
''' mean mu  '''
binlen = 6
fig,ax2 = plt.subplots(2,1,figsize=(4,4))
positions = np.arange(0, binlen*ngavg-1, binlen)
add_label(ax2[0].violinplot(
        np.squeeze(np.abs(mu_x_num[:,:,0])).T, positions, showmeans=False, showmedians=True,
        showextrema=False), "num. Full")   
add_label(ax2[1].violinplot(
        np.squeeze(np.abs(mu_x_num[:,:,1])).T, positions, showmeans=False, showmedians=True,
        showextrema=False), "num. Full")    


positions = np.arange(1, binlen*ngavg-1, binlen)
add_label(ax2[0].violinplot(
        np.squeeze(np.abs(mu_kappa_num[:,:,0])).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"num. Rank-one")
add_label(ax2[1].violinplot(
        np.squeeze(np.abs(mu_kappa_num[:,:,1])).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"num. Rank-one")

positions = np.arange(3, binlen*ngavg-1, binlen)
add_label(ax2[0].violinplot(
        np.squeeze(mu_full_theo[:,:,0]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"theo. Full")
add_label(ax2[1].violinplot(
        np.squeeze(mu_full_theo[:,:,1]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"theo. Full")

positions = np.arange(4, binlen*ngavg-1, binlen)
add_label(ax2[0].violinplot(
        np.squeeze(mu_R1_theo[:,:,0]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"theo. Rank-one")
add_label(ax2[1].violinplot(
        np.squeeze(mu_R1_theo[:,:,1]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"theo. Rank-one")



### WAY ONE --- for variance
labels = []
fig,ax2 = plt.subplots(2,1,figsize=(4,4))
positions = np.arange(0, binlen*ngavg-1, binlen)
add_label(ax2[0].violinplot(
        np.squeeze(variance_x_num[:,:,0]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), "num. Full")   
add_label(ax2[1].violinplot(
        np.squeeze(variance_x_num[:,:,1]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), "num. Full")    

positions = np.arange(1, binlen*ngavg-1, binlen)
add_label(ax2[0].violinplot(
        np.squeeze(variance_kappa_num[:,:,0]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"num. Rank-one")
add_label(ax2[1].violinplot(
        np.squeeze(variance_kappa_num[:,:,1]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"num. Rank-one")

positions = np.arange(3, binlen*ngavg-1, binlen)
add_label(ax2[0].violinplot(
        np.squeeze(variance_full_theo[:,:,0]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"theo. Full")
add_label(ax2[1].violinplot(
        np.squeeze(variance_full_theo[:,:,1]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"theo. Full")

positions = np.arange(4, binlen*ngavg-1, binlen)
add_label(ax2[0].violinplot(
        np.squeeze(variance_R1_theo[:,:,0]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"theo. Rank-one")
add_label(ax2[1].violinplot(
        np.squeeze(variance_R1_theo[:,:,1]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"theo. Rank-one")


In [ ]:
## Figures reflecting how mean and variance change with random gain of iid Gaussian matrix
fig,ax2 = plt.subplots(1,2,figsize=(4,2))
ax2.plot(gavgseries,mu_full_theo[:,0,0],color='darkred',linewidth= 1.5,label=r'$\mu_F^E$')
ax2.plot(gavgseries,mu_full_theo[:,0,1],color='red',linewidth= 1.5,label=r'$\mu_F^I$')
ax2.plot(gavgseries,mu_R1_theo[:,0,0],color='darkblue',linewidth= 1.5,label=r'$\mu_{R1}^E$')
ax2.plot(gavgseries,mu_R1_theo[:,0,1],color='blue',linewidth= 1.5,label=r'$\mu_{R1}^I$')
plt.legend()

fig,ax2 = plt.subplots(figsize=(4,2))
ax2.plot(gavgseries,variance_full_theo[:,0,0],color='darkred',linewidth= 1.5,label=r'$\Delta_F^E$')
ax2.plot(gavgseries,variance_full_theo[:,0,1],color='red',linewidth= 1.5,label=r'$\Delta_F^I$')
ax2.plot(gavgseries,variance_R1_theo[:,0,0],color='darkblue',linewidth= 1.5,label=r'$\Delta_{R1}^E$')
ax2.plot(gavgseries,variance_R1_theo[:,0,1],color='blue',linewidth= 1.5,label=r'$\Delta_{R1}^I$')
plt.legend()

In [ ]:
## Figures reflecting how mean and variance change with random gain of iid Gaussian matrix
fig,ax2 = plt.subplots(2,1,figsize=(6,6))
for iktrial in range(ntrial):
    ax2[0].scatter(gavgseries-0.01,np.abs(mu_x_num[:,iktrial,0]),s=8.0,color='red',edgecolors='red',marker='^',alpha=0.5)
    ax2[1].scatter(gavgseries-0.01,np.abs(mu_x_num[:,iktrial,1]),s=8.0,color='blue',edgecolors='blue',marker='^',alpha=0.5)
    ax2[0].scatter(gavgseries+0.01,np.abs(mu_kappa_num[:,iktrial,0]),s=8.0,color='orange',edgecolors='orange',marker='s',alpha=0.5)
    ax2[1].scatter(gavgseries+0.01,np.abs(mu_kappa_num[:,iktrial,1]),s=8.0,color='green',edgecolors='green',marker='s',alpha=0.5)

    ax2[0].scatter(gavgseries,np.abs(mu_gavg_num[:,iktrial,0]),s=8.0,color='black',edgecolors='black',marker='o',alpha=0.5)
    ax2[1].scatter(gavgseries,np.abs(mu_gavg_num[:,iktrial,1]),s=8.0,color='black',edgecolors='black',marker='o',alpha=0.5)

ax2[0].plot(gavgseries,np.mean(mu_full_theo[:,:,0],axis=1),color='red',linewidth=1.5,label=r'$\mu_F^E$',alpha=0.5)
ax2[1].plot(gavgseries,np.mean(mu_full_theo[:,:,1],axis=1),color='blue',linewidth=1.5,label=r'$\mu_F^I$',alpha=0.5)
ax2[0].plot(gavgseries,np.mean(mu_R1_theo[:,:,0],axis=1),color='orange',linewidth=1.5,label=r'$\mu_{R1}^E$',alpha=0.5)
ax2[1].plot(gavgseries,np.mean(mu_R1_theo[:,:,1],axis=1),color='green',linewidth=1.5,label=r'$\mu_{R1}^I$',alpha=0.5)
xticks = np.linspace(.0,0.8,2)
xlims = [-0.1,0.9]
yticks = np.linspace(.0,1.0,2)
ylims = [0.0,1.2]

for i in range(2):     
        ax2[i].set_xlim(xlims)
        ax2[i].set_ylim(ylims)
        ax2[i].set_xticks(xticks)
        ax2[i].set_yticks(yticks)
        ax2[i].legend()


fig,ax2 = plt.subplots(2,1,figsize=(6,6))
for iktrial in range(ntrial):
    ax2[0].scatter(gavgseries-0.01,variance_x_num[:,iktrial,0],s=8.0,color='red',edgecolors='red',marker='^',alpha=0.5)
    ax2[1].scatter(gavgseries-0.01,variance_x_num[:,iktrial,1],s=8.0,color='blue',edgecolors='blue',marker='^',alpha=0.5)
    ax2[0].scatter(gavgseries+0.01,variance_kappa_num[:,iktrial,0],s=8.0,color='orange',edgecolors='orange',marker='s',alpha=0.5)
    ax2[1].scatter(gavgseries+0.01,variance_kappa_num[:,iktrial,1],s=8.0,color='green',edgecolors='green',marker='s',alpha=0.5)

    ax2[0].scatter(gavgseries,variance_gavg_num[:,iktrial,0],s=8.0,color='black',edgecolors='black',marker='o',alpha=0.5)
    ax2[1].scatter(gavgseries,variance_gavg_num[:,iktrial,1],s=8.0,color='black',edgecolors='black',marker='o',alpha=0.5)


ax2[0].plot(gavgseries,np.mean(variance_full_theo[:,:,0],axis=1),color='red',linewidth=1.5,label=r'$\Delta_F^E$',alpha=0.5)
ax2[1].plot(gavgseries,np.mean(variance_full_theo[:,:,1],axis=1),color='blue',linewidth=1.5,label=r'$\Delta_F^I$',alpha=0.5)
ax2[0].plot(gavgseries,np.mean(variance_R1_theo[:,:,0],axis=1),color='orange',linewidth=1.5,label=r'$\Delta_{R1}^E$',alpha=0.5)
ax2[1].plot(gavgseries,np.mean(variance_R1_theo[:,:,1],axis=1),color='green',linewidth=1.5,label=r'$\Delta_{R1}^I$',alpha=0.5)

xticks = np.linspace(.0,0.8,2)
xlims = [-0.1,0.9]
yticks = np.linspace(.0,0.3,2)
ylims = [-.05,0.35]

for i in range(2):     
        ax2[i].set_xlim(xlims)
        ax2[i].set_ylim(ylims)
        ax2[i].set_xticks(xticks)
        ax2[i].set_yticks(yticks)
        ax2[i].legend()

* change with $J_E$

In [ ]:
'''
WITHOUT SYMMETRY, 
* increase JE to change JE-JI<1 to JE-JI>1
* constant g0 # randomness
'''
from functools import partial
# generate mean matrix
nrank=1
Nt=np.array([300,300])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
nrank,ntrial,neta,nvec,nmaxFP=1,30,1,2,3
nJE = 3
JEseries = np.linspace(1.65,2.65,nJE)
''' Network Setting  '''
## heterogeneous random gain
gaverage = 0.8
xee,xei,xie,xii=1.0,1.0,1.0,1.0 

## heterogeneous degree of symmetry: amplitudes and signs
coeffetaEsub=np.array([1.0,1.0,1.0])#
coeffetaTotal = np.array([1.0,1.0,1.0])#
# coeffetaTotal = np.zeros(3)
signetaEsub=np.ones(3)
signetaTotal=np.ones(3)

ppercentEsub = np.ones(2)
ppercentEsub[0]=0.5
ppercentEsub[1]=1.0-ppercentEsub[0]
## E->total ppercentEsub[0]/2.0,ppercentEsub[1]/2.0, (I) 1/2.0

'''  Recording Variables. '''
## perturbed + symmetric eigenvectors
Reigvecseries,Leigvecseries=np.zeros((nJE,ntrial,NE*2,nvec*2),dtype=complex),np.zeros((nJE,ntrial,NE*2,nvec*2),dtype=complex)
ReigvecTseries,LeigvecTseries=np.zeros((nJE,ntrial,NE*2,nvec*2),dtype=complex),np.zeros((nJE,ntrial,NE*2,nvec*2),dtype=complex)
Beigvseries=np.zeros((nJE,ntrial,N),dtype=complex)
## corresponding statistics \mu and \sigma \cov
armu,sigrcov = np.zeros((nJE,ntrial,2),dtype=complex),np.zeros((nJE,ntrial,2),dtype=complex) # 2 for E and I
almu,siglcov = np.zeros((nJE,ntrial,2),dtype=complex),np.zeros((nJE,ntrial,2),dtype=complex)
siglr = np.zeros((nJE,ntrial,2),dtype=complex)
## reference, eigenvectors of matrix with iid perturbations
x0series,y0series=np.zeros((nJE,ntrial,NE*2),dtype=complex),np.zeros((nJE,ntrial,NE*2),dtype=complex)
gYbasexee=np.zeros((ntrial,NE*2,NE*2),dtype=complex)

# ## perturbed + symmetric eigenvectors
# Reigvecseries,Leigvecseries=np.zeros((nJE,ntrial,NE*2,nvec*2)),np.zeros((nJE,ntrial,NE*2,nvec*2))
# ReigvecTseries,LeigvecTseries=np.zeros((nJE,ntrial,NE*2,nvec*2)),np.zeros((nJE,ntrial,NE*2,nvec*2))
# Beigvseries=np.zeros((nJE,ntrial,N))
# ## corresponding statistics \mu and \sigma \cov
# armu,sigrcov = np.zeros((nJE,ntrial,2)),np.zeros((nJE,ntrial,2)) # 2 for E and I
# almu,siglcov = np.zeros((nJE,ntrial,2)),np.zeros((nJE,ntrial,2))
# siglr = np.zeros((nJE,ntrial,2))
# ## reference, eigenvectors of matrix with iid perturbations
# x0series,y0series=np.zeros((nJE,ntrial,NE*2)),np.zeros((nJE,ntrial,NE*2))
# gYbasexee=np.zeros((ntrial,NE*2,NE*2))

''' Nullclines parameters  '''
kappaintersect_Full = np.zeros((nJE,ntrial,nmaxFP*2))
kappaintersect_R1   = np.zeros((nJE,ntrial,nmaxFP*2))

''' simulating parameters '''
tt = np.linspace(0,150,500)
dt = tt[2]-tt[1]
ntt = len(tt)
xfpseries_Full = np.zeros((nJE,ntrial,NE*2,ntt))
xfpseries_R1   = np.zeros((nJE,ntrial,NE*2,ntt))
''' All random matrices for each trials '''

''' Iterative Processing '''
for iktrial in range(ntrial):
    print('>>>>>>> simulating neuronal activity ... >>>>>>')
    '''    ##>>>>>>>>>>> g0!=0, >>>>>>>>     '''
    Xsym =iidGaussian([0,gaverage/np.sqrt(NE*2)],[NE*2,NE*2])
    XsymT=Xsym.copy().T
    X0=Xsym.copy()
    ## log: there is a bug, X is the total random xconn, including symmetry
    gYbasexee[iktrial,:,:]=Xsym.copy()

    for idxje,JEv in enumerate(JEseries):
        ''' ## Three \bar{J} cases '''
        JI,JE,a,b=0.5,JEv,0,0
        JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
        ''' Am -- J(g0=0), xAm(g0=0), yAm(g0=0) '''
        Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
        ''' for reference, eigenvectors of unperturbed matrix \bar{J}'''
        meigvecAm,neigvecAm,eigvAm = decompunperturbedEI(Am)
        xAm,yAm=np.reshape(meigvecAm[:,0],(NE*2,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(NE*2,1))
        eta = 0
        etaset=eta*np.ones(6)
        for iloop in range(3):
            etaset[iloop] = etaset[iloop]*coeffetaEsub[iloop]
            etaset[iloop+3] = etaset[iloop+3]*coeffetaTotal[iloop]
        Xinit = Xsym.copy()            
        X=Xinit.copy()
        X[:NE,:NE]*=(xee)
        X[:NE,NE:]*=(xei)
        X[NE:,:NE]*=(xie)
        X[NE:,NE:]*=(xii)

        # overall
        J = X.copy()+Am.copy()
        JT = J.copy().T
        ''' iid Gaussian randomness '''
        eigvJ,leigvec,reigvec,xnorm0,ynorm0=decompNormalization(J,xAm,yAm,sort=0)
        ''' ## P = mn.T/N, already have rl.T -- r(m) \lambda l.T(n.T) -- Reig*\sqrt(N)Leig*\sqrt(N)/N   '''
        xnorm0,ynorm0 = xnorm0*np.sqrt(2*NE),ynorm0*np.sqrt(2*NE)
        Reigvecseries[idxje,iktrial,:,0],Leigvecseries[idxje,iktrial,:,0]=xnorm0[:,0].copy(),ynorm0[:,0].copy()
        Beigvseries[idxje,iktrial,:]=eigvJ.copy()

        axrmu,aylmu,sigxr,sigyl,sigcov = numerical_stats(xnorm0,ynorm0,xAm*np.sqrt(NE*2),yAm*np.sqrt(NE*2),eigvJ,nrank,2,ppercent=np.array([0.5,0.5]))
        armu[idxje,iktrial,:],almu[idxje,iktrial,:] = axrmu[:,0],aylmu[:,0]
        sigrcov[idxje,iktrial,:],siglcov[idxje,iktrial,:]= sigxr[:,0],sigyl[:,0]
        siglr[idxje,iktrial,:] = sigcov[:,0,0]
        
        ## Rank One Approximation, Nullclines of \kappa
        xnorm0, ynorm0 = Reigvecseries[idxje,iktrial,:,0].copy(),Leigvecseries[idxje,iktrial,:,0]
        xnorm0, ynorm0 = np.reshape(xnorm0,(2*NE,1)),np.reshape(ynorm0,(2*NE,1))
        xmu,ymu = armu[idxje,iktrial,:].copy(),almu[idxje,iktrial,:].copy()
        xsig,ysig = sigrcov[idxje,iktrial,:].copy(),siglcov[idxje,iktrial,:].copy()
        yxcov = siglr[idxje,iktrial,:].copy()

        ''' Full Connectivity -- Dynamics '''
        Jpt = J.copy()
        xinit = np.random.normal(0,1E-2,(1,NE*2))
        xinit = np.squeeze(xinit)
        xtemporal = odesimulation(tt,xinit,Jpt,0)
        xfpseries_Full[idxje,iktrial,:,:] = xtemporal.T.copy()
        ## 2 KAPPA_0
        kappanum = np.zeros(3)
        xact = np.squeeze(xfpseries_Full[idxje,iktrial,:,-1])
        ## use yAm -- unperturbed centre.
        if((ymu[0]*eigvJ[0])*(yAm[0,0]*eigvAm[0])>0):
            kappanum[0] = yAm[0,0]*np.mean(np.tanh(xact[:NE]))/2.0+yAm[NE,0]*np.mean(np.tanh(xact[NE:]))/2.0
        else:
            kappanum[0] = -yAm[0,0]*np.mean(np.tanh(xact[:NE]))/2.0-yAm[NE,0]*np.mean(np.tanh(xact[NE:]))/2.0
        kappanum[0]=kappanum[0]*np.sqrt(NE*2)*eigvAm[0]
        kappaintersect_Full[idxje,iktrial,:3]=kappanum[:].copy()

        ''' use perturbation theorem to calculate the perturbed vectors ''' 
        xnormt, ynormt = X.copy()@xAm.copy()/eigvAm[0], X.copy().T@yAm.copy()/eigvAm[0]
        xnormt, ynormt = xnormt+xAm.copy(), ynormt+yAm.copy()
        ## normalize 
        xnormt = xnormt.copy()/np.linalg.norm(xnormt.copy())
        ynormt = ynormt.copy()
        _,_,xnormt,ynormt=Normalization(xnormt.copy(),ynormt.copy(),xAm.copy(),yAm.copy(),sort=0,nrank=1)
        ReigvecTseries[idxje,iktrial,:,0],LeigvecTseries[idxje,iktrial,:,0]=xnormt[:,0].copy(),ynormt[:,0].copy()
        xnormt,ynormt = xnormt*np.sqrt(NE*2),ynormt*np.sqrt(NE*2)

        # xnormt, ynormt = X.copy()@xAm.copy()/eigvAm[0]*np.sqrt(NE*2), X.copy().T@yAm.copy()/eigvAm[0]*np.sqrt(NE*2)#yAm.copy().T@X/eigvAm[0]*np.sqrt(N)
        # xnormt, ynormt = xnormt+xAm.copy()*np.sqrt(NE*2), ynormt+yAm.copy()*np.sqrt(NE*2)
        # ReigvecTseries[idxje,iktrial,:,0],LeigvecTseries[idxje,iktrial,:,0]=xnormt[:,0].copy(),ynormt[:,0].copy()
        r1Matt = np.real(xnormt@ynormt.T)
        r1Matt = r1Matt*np.real(eigvAm[0])/NE/2.0
        ### START SIMULATING AND CALCULATE KAPPA AND xfpsereis_R1
        ## use the same initial values
        xtemporal = odesimulation(tt,xinit,r1Matt,0)
        xfpseries_R1[idxje,iktrial,:,:] = xtemporal.T.copy()
        ''' kappa dynamics    '''
        ## 2 populations 
        kappanum = np.zeros(3)
        xact = np.squeeze(xfpseries_R1[idxje,iktrial,:,-1])
        ## use yAm -- unperturbed centre.
        if((ymu[0]*eigvJ[0])*(yAm[0,0]*eigvAm[0])>0):
            # do not change
            kappanum[0] = yAm[0,0]*np.mean(np.tanh(xact[:NE]))/2.0+yAm[NE,0]*np.mean(np.tanh(xact[NE:]))/2.0
        else:
            kappanum[0] = -yAm[0,0]*np.mean(np.tanh(xact[:NE]))/2.0-yAm[NE,0]*np.mean(np.tanh(xact[NE:]))/2.0
        kappanum[0]=kappanum[0]*np.sqrt(NE*2)*eigvAm[0]
        kappaintersect_R1[idxje,iktrial,:3]=kappanum[:].copy()

        # ''' Rank one Connectivity Approximation -- Dynamics '''
        # r1Mat = xnorm0.copy()@ynorm0.copy().T
        # normeigV = xnorm0.copy().T@ynorm0.copy()
        # normeigV = np.squeeze(normeigV)
        # r1Mat = (r1Mat*np.real(eigvAm[0])/NE/2)
        # ### START SIMULATING AND CALCULATE KAPPA AND xfpsereis_R1
        # ## use the same initial values
        # xtemporal = odesimulation(tt,xinit,r1Mat,0)
        # xfpseries_R1[idxje,iktrial,:,:] = xtemporal.T.copy()
        # ''' kappa dynamics    '''
        # ## 2 populations 
        # kappanum = np.zeros(3)
        # xact = np.squeeze(xfpseries_R1[idxje,iktrial,:,-1])
        # ## use yAm -- unperturbed centre.
        # if((ymu[0]*eigvJ[0])*(yAm[0,0]*eigvAm[0])>0):
        #     # do not change
        #     kappanum[0] = yAm[0,0]*np.mean(np.tanh(xact[:NE]))/2.0+yAm[NE,0]*np.mean(np.tanh(xact[NE:]))/2.0
        # else:
        #     kappanum[0] = -yAm[0,0]*np.mean(np.tanh(xact[:NE]))/2.0-yAm[NE,0]*np.mean(np.tanh(xact[NE:]))/2.0
        # kappanum[0]=kappanum[0]*np.sqrt(NE*2)*eigvAm[0]
        # kappaintersect_R1[idxje,iktrial,:3]=kappanum[:].copy()



In [ ]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE
Randgain=np.zeros(4)
Randgain[0],Randgain[1],Randgain[2],Randgain[3]=xee,xei,xie,xii

np.savez(now+'_iidDynamics_JE_data',Jparameters=Jparameters,JEseries=JEseries,Randgain=Randgain,gaverage=gaverage,Reigvecseries=Reigvecseries[:,:,:,0],Leigvecseries=Leigvecseries[:,:,:,0],ReigvecTseries=ReigvecTseries[:,:,:,0],LeigvecTseries=LeigvecTseries[:,:,:,0],Beigvseries=Beigvseries[:,:,0],gYbasexee=gYbasexee,kappaintersect_Full=kappaintersect_Full,kappaintersect_R1=kappaintersect_R1,xfpseries_Full=xfpseries_Full[:,:,:,-1],xfpseries_R1=xfpseries_R1[:,:,:,-1])

In [ ]:
## how did the mean and variance co-change
xticks = np.linspace(.0,0.5,2)
xlims = [-.05,0.55]
yticks = np.linspace(.0,0.4,2)
ylims = [-.05,0.4]
nsigx2 = 20
idxje = 2
sigx2set= np.linspace(0.0,0.5,nsigx2)
mucorrset = np.zeros(nsigx2)
for i in range(nsigx2):
    muini = 4
    mucorrset[i]= fsolve(iid_muCdelta_consistency,muini,args=(JEseries[idxje],JI,gaverage,sigx2set[i]))
fig,ax = plt.subplots(figsize=(3,2))
mucorrset-=mucorrset[-1]
ax.plot(sigx2set,mucorrset,linewidth=1.5)#color='black',
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
ax.axvline(c='grey', lw=1)
ax.axhline(c='grey', lw=1)

## using self-consistency in kappa to solve the dynamics of rank one approximated network (as to full conn. should use self-consistency in mean and variance)
kappa_theo_iid = np.zeros((nJE,3))
for idxje in range(nJE):
    init_k=1.5
    kappa_theo_iid[idxje,0]= fsolve(iidperturbation,init_k,args=(JEseries[idxje],JI,gaverage))
    init_k=0.0
    kappa_theo_iid[idxje,1]= fsolve(iidperturbation,init_k,args=(JEseries[idxje],JI,gaverage))
    init_k=-1.5
    kappa_theo_iid[idxje,2]= fsolve(iidperturbation,init_k,args=(JEseries[idxje],JI,gaverage))
    
## plot kappa change with JE ##
fig,ax = plt.subplots(figsize=(4,4))
ax.plot(JEseries,kappa_theo_iid[:,0],color='black',linewidth=1.5)
ax.plot(JEseries,kappa_theo_iid[:,1],color='gray',linewidth=1.5)
ax.plot(JEseries,kappa_theo_iid[:,2],color='black',linewidth=1.5)
for iktrial in range(ntrial):
    ax.plot(JEseries,kappaintersect_Full[:,iktrial,0],'^',color='blue',markersize=5)
    ax.plot(JEseries,kappaintersect_R1[:,iktrial,0],'^',color='green',markersize=5)

In [ ]:
## test two expressions of variance ##
variance_x_num,variance_kappa_num = np.zeros((nJE,ntrial,2)),np.zeros((nJE,ntrial,2))
mu_x_num,mu_kappa_num = np.zeros((nJE,ntrial,2)),np.zeros((nJE,ntrial,2))
variance_full_theo,variance_R1_theo = np.zeros((nJE,ntrial,2)),np.zeros((nJE,ntrial,2))
mu_full_theo, mu_R1_theo = np.zeros((nJE,ntrial,2)),np.zeros((nJE,ntrial,2))
for idxje in range(nJE):
    for iktrial in range(ntrial):
        ## numerical results for Full Mat
        variance_x_num[idxje,iktrial,0],variance_x_num[idxje,iktrial,1]=np.std(xfpseries_Full[idxje,iktrial,:NE,-1])**2,np.std(xfpseries_Full[idxje,iktrial,NE:,-1])**2
        mu_x_num[idxje,iktrial,0],mu_x_num[idxje,iktrial,1]=np.mean(xfpseries_Full[idxje,iktrial,:NE,-1]),np.mean(xfpseries_Full[idxje,iktrial,NE:,-1])
        ## numerical results for Rank one Appriximation Mat
        variance_kappa_num[idxje,iktrial,0],variance_kappa_num[idxje,iktrial,1]=np.std(xfpseries_R1[idxje,iktrial,:NE,-1])**2,np.std(xfpseries_R1[idxje,iktrial,NE:,-1])**2
        mu_kappa_num[idxje,iktrial,0],mu_kappa_num[idxje,iktrial,1]=np.mean(xfpseries_R1[idxje,iktrial,:NE,-1]),np.mean(xfpseries_R1[idxje,iktrial,NE:,-1])
        
for idxje in range(nJE):
    # ## theoretical    
    deltainit,meaninit = np.mean(variance_x_num[idxje,:,0]),kappa_theo_iid[idxje,0]#np.mean(kappaintersect[idxje,:,0])
    statsfull = fsolve(iidfull_mudelta_consistency,[meaninit,deltainit],args=(JEseries[idxje],JI,gaverage))
    mu_full_theo[idxje,:,0],variance_full_theo[idxje,:,0]= statsfull[0],statsfull[1]*np.ones(ntrial)
    mu_full_theo[idxje,:,1],variance_full_theo[idxje,:,1]= statsfull[0],statsfull[1]*np.ones(ntrial)
    # statsfull = fsolve(iidR1_mudelta_consistency,[meaninit,deltainit],args=(JEseries[idxje],JI,gaverage))
    # mu_R1_theo[idxje,0],variance_R1_theo[idxje,:,0]= statsfull[0],statsfull[1]*np.ones(ntrial)
    # mu_R1_theo[idxje,1],variance_R1_theo[idxje,:,1]= statsfull[0],statsfull[1]*np.ones(ntrial)
    mu_R1_theo[idxje,:,0],variance_R1_theo[idxje,:,0]= kappa_theo_iid[idxje,0],kappa_theo_iid[idxje,0]**2*gaverage**2/(JEseries[idxje]-JI)**2
    mu_R1_theo[idxje,:,1],variance_R1_theo[idxje,:,1]= kappa_theo_iid[idxje,0],kappa_theo_iid[idxje,0]**2*gaverage**2/(JEseries[idxje]-JI)**2
    ## There is an error for I use JE here

import matplotlib.patches as mpatches
### WAY ONE --- for variance
xticks = np.linspace(.0,0.60,2)
xlims = [.0,0.65]
yticks = np.linspace(.0,0.60,2)
ylims = [.0,0.65]

vmeanxnum,vmeankappanum = np.zeros((nJE,2)),np.zeros((nJE,2))
mmeanxnum,mmeankappanum = np.zeros((nJE,2)),np.zeros((nJE,2))
for i in range(2):
        vmeanxnum[:,i] = np.mean(variance_x_num[:,:,i],axis=1)
        vmeankappanum[:,i] = np.mean(variance_kappa_num[:,:,i],axis=1)
        mmeanxnum[:,i] = np.mean(mu_x_num[:,:,i],axis=1)
        mmeankappanum[:,i] = np.mean(mu_kappa_num[:,:,i],axis=1)

fig,ax2 = plt.subplots(1,2,figsize=(6,4))
positions = np.arange(0, 5*nJE-1, 5)
for iktrial in range(ntrial):
        ax2[0].plot(variance_x_num[:,iktrial,0],variance_kappa_num[:,iktrial,0],'^',markersize=1,color='red')
        ax2[1].plot(variance_x_num[:,iktrial,1],variance_kappa_num[:,iktrial,1],'^',markersize=1,color='blue')
        # ax2[0].plot(variance_gavg_num[:,iktrial,0],variance_gkappa_num[:,iktrial,0],'o',markersize=1,color='orange')
        # ax2[1].plot(variance_gavg_num[:,iktrial,1],variance_gkappa_num[:,iktrial,1],'o',markersize=1,color='green')
        ax2[0].plot(variance_full_theo[:,iktrial,0],variance_R1_theo[:,iktrial,0],'o',markersize=5,color='orange')#'black')
        ax2[1].plot(variance_full_theo[:,iktrial,0],variance_R1_theo[:,iktrial,0],'o',markersize=5,color='green')#'black')

xxx = np.linspace(0.0,0.50,2)

for i in range(2):
        varnum = np.polyfit(np.squeeze(variance_x_num[1:,:,i]).T.flatten(), np.squeeze(variance_kappa_num[1:,:,i]).T.flatten(), deg=1)
        # yvar=varnum[0]*xxx+varnum[1]
        # ax2[i].plot(xxx,yvar,linestyle='-',color='gray')   
        ax2[i].plot(xxx,xxx,linestyle='--',color='black')             
        ax2[i].set_xlim(xlims)
        ax2[i].set_ylim(ylims)
        ax2[i].set_xticks(xticks)
        ax2[i].set_yticks(yticks)
        ax2[i].set_aspect('equal')


xticks = np.linspace(.0,2.0,2)
xlims = [-.25,2.25]
yticks = np.linspace(.0,2.0,2)
ylims = [-.25,2.25]
fig,ax2 = plt.subplots(1,2,figsize=(6,4))
positions = np.arange(0, 5*nJE-1, 5)
medianxtheo,mediankappatheo = np.zeros((nJE,2)),np.zeros((nJE,2))
for iktrial in range(ntrial):
        ax2[0].plot(np.abs(mu_x_num[:,iktrial,0]),np.abs(mu_kappa_num[:,iktrial,0]),'^',markersize=1,color='red')
        ax2[1].plot(np.abs(mu_x_num[:,iktrial,1]),np.abs(mu_kappa_num[:,iktrial,1]),'^',markersize=1,color='blue')
        ax2[0].plot(mu_full_theo[:,iktrial,0],mu_R1_theo[:,iktrial,0],'o',markersize=5,color='orange')#'black')
        ax2[1].plot(mu_full_theo[:,iktrial,0],mu_R1_theo[:,iktrial,0],'o',markersize=5,color='green')#'black')

xxx = np.linspace(0.0,1.50,2)
for i in range(2):
        # munum = np.polyfit(np.squeeze(mu_x_num[:,:,i]).T.flatten(), np.squeeze(mu_kappa_num[:,:,i]).T.flatten(), deg=1)
        # ymu=munum[0]*xxx+munum[1]
        # ax2[i].plot(xxx,ymu,linestyle='-',color='gray')  

        ax2[i].plot(xxx,xxx,linestyle='--',color='black') 
        ax2[i].set_xlim(xlims)
        ax2[i].set_ylim(ylims)
        ax2[i].set_xticks(xticks)
        ax2[i].set_yticks(yticks)
        ax2[i].set_aspect('equal')


In [ ]:
def add_label(violin, label):
    color = violin["bodies"][0].get_facecolor().flatten()
    labels.append((mpatches.Patch(color=color), label))
labels = []
''' mean mu  '''
binlen = 6
fig,ax2 = plt.subplots(2,1,figsize=(4,4))
positions = np.arange(0, binlen*nJE-1, binlen)
add_label(ax2[0].violinplot(
        np.squeeze(np.abs(mu_x_num[:,:,0])).T, positions, showmeans=False, showmedians=True,
        showextrema=False), "num. Full")   
add_label(ax2[1].violinplot(
        np.squeeze(np.abs(mu_x_num[:,:,1])).T, positions, showmeans=False, showmedians=True,
        showextrema=False), "num. Full")    


positions = np.arange(1, binlen*nJE-1, binlen)
add_label(ax2[0].violinplot(
        np.squeeze(np.abs(mu_kappa_num[:,:,0])).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"num. Rank-one")
add_label(ax2[1].violinplot(
        np.squeeze(np.abs(mu_kappa_num[:,:,1])).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"num. Rank-one")

positions = np.arange(3, binlen*nJE-1, binlen)
add_label(ax2[0].violinplot(
        np.squeeze(mu_full_theo[:,:,0]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"theo. Full")
add_label(ax2[1].violinplot(
        np.squeeze(mu_full_theo[:,:,1]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"theo. Full")

positions = np.arange(4, binlen*nJE-1, binlen)
add_label(ax2[0].violinplot(
        np.squeeze(mu_R1_theo[:,:,0]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"theo. Rank-one")
add_label(ax2[1].violinplot(
        np.squeeze(mu_R1_theo[:,:,1]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"theo. Rank-one")

### WAY ONE --- for variance
labels = []
fig,ax2 = plt.subplots(2,1,figsize=(4,4))
positions = np.arange(0, binlen*nJE-1, binlen)
add_label(ax2[0].violinplot(
        np.squeeze(variance_x_num[:,:,0]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), "num. Full")   
add_label(ax2[1].violinplot(
        np.squeeze(variance_x_num[:,:,1]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), "num. Full")    

positions = np.arange(1, binlen*nJE-1, binlen)
add_label(ax2[0].violinplot(
        np.squeeze(variance_kappa_num[:,:,0]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"num. Rank-one")
add_label(ax2[1].violinplot(
        np.squeeze(variance_kappa_num[:,:,1]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"num. Rank-one")

positions = np.arange(3, binlen*nJE-1, binlen)
add_label(ax2[0].violinplot(
        np.squeeze(variance_full_theo[:,:,0]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"theo. Full")
add_label(ax2[1].violinplot(
        np.squeeze(variance_full_theo[:,:,1]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"theo. Full")

positions = np.arange(4, binlen*nJE-1, binlen)
add_label(ax2[0].violinplot(
        np.squeeze(variance_R1_theo[:,:,0]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"theo. Rank-one")
add_label(ax2[1].violinplot(
        np.squeeze(variance_R1_theo[:,:,1]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"theo. Rank-one")

# positions = np.arange(2, 7*nJE-1, 7)
# add_label(ax2[0].violinplot(
#         np.squeeze(variance_gavg_num[:,:,0]).T, positions, showmeans=False, showmedians=True,
#         showextrema=False), r"$g_0^2\langle\phi^{E2}\rangle$")
# add_label(ax2[1].violinplot(
#         np.squeeze(variance_gavg_num[:,:,1]).T, positions, showmeans=False, showmedians=True,
#         showextrema=False), r"$g_0^2\langle\phi^{I2}\rangle$")

# positions = np.arange(3, 7*nJE-1, 7)
# add_label(ax2[0].violinplot(
#         np.squeeze(variance_gkappa_num[:,:,0]).T, positions, showmeans=False, showmedians=True,
#         showextrema=False), r"$g_0^2\langle\phi^{E}\rangle^2$")
# add_label(ax2[1].violinplot(
#         np.squeeze(variance_gkappa_num[:,:,1]).T, positions,showmeans=False, showmedians=True,
#         showextrema=False), r"$g_0^2\langle\phi^{I}\rangle^2$")

In [11]:
## How the statistics of populational dynamics change, with respect to the excitatory connection $J_E$
## Figures reflecting how mean and variance change with random gain of iid Gaussian matrix
fig,ax2 = plt.subplots(2,1,figsize=(6,6))
for iktrial in range(ntrial):
    ax2[0].scatter(JEseries-0.01,np.abs(mu_x_num[:,iktrial,0]),s=8.0,color='red',edgecolors='red',marker='^',alpha=0.5)
    ax2[1].scatter(JEseries-0.01,np.abs(mu_x_num[:,iktrial,1]),s=8.0,color='blue',edgecolors='blue',marker='^',alpha=0.5)
    ax2[0].scatter(JEseries+0.01,np.abs(mu_kappa_num[:,iktrial,0]),s=8.0,color='orange',edgecolors='orange',marker='s',alpha=0.5)
    ax2[1].scatter(JEseries+0.01,np.abs(mu_kappa_num[:,iktrial,1]),s=8.0,color='green',edgecolors='green',marker='s',alpha=0.5)


ax2[0].plot(JEseries,np.mean(mu_full_theo[:,:,0],axis=1),color='red',linewidth=1.5,label=r'$\mu_F^E$',alpha=0.5)
ax2[1].plot(JEseries,np.mean(mu_full_theo[:,:,1],axis=1),color='blue',linewidth=1.5,label=r'$\mu_F^I$',alpha=0.5)
ax2[0].plot(JEseries,np.mean(mu_R1_theo[:,:,0],axis=1),color='orange',linewidth=1.5,label=r'$\mu_{R1}^E$',alpha=0.5)
ax2[1].plot(JEseries,np.mean(mu_R1_theo[:,:,1],axis=1),color='green',linewidth=1.5,label=r'$\mu_{R1}^I$',alpha=0.5)
xticks = np.linspace(.0,0.8,2)
xlims = [-0.1,0.9]
yticks = np.linspace(.0,1.0,2)
ylims = [0.0,1.2]

for i in range(2):     
        # ax2[i].set_xlim(xlims)
        # ax2[i].set_ylim(ylims)
        # ax2[i].set_xticks(xticks)
        # ax2[i].set_yticks(yticks)
        ax2[i].legend()


fig,ax2 = plt.subplots(2,1,figsize=(6,6))
for iktrial in range(ntrial):
    ax2[0].scatter(JEseries-0.01,variance_x_num[:,iktrial,0],s=8.0,color='red',edgecolors='red',marker='^',alpha=0.5)
    ax2[1].scatter(JEseries-0.01,variance_x_num[:,iktrial,1],s=8.0,color='blue',edgecolors='blue',marker='^',alpha=0.5)
    ax2[0].scatter(JEseries+0.01,variance_kappa_num[:,iktrial,0],s=8.0,color='orange',edgecolors='orange',marker='s',alpha=0.5)
    ax2[1].scatter(JEseries+0.01,variance_kappa_num[:,iktrial,1],s=8.0,color='green',edgecolors='green',marker='s',alpha=0.5)


ax2[0].plot(JEseries,np.mean(variance_full_theo[:,:,0],axis=1),color='red',linewidth=1.5,label=r'$\Delta_F^E$',alpha=0.5)
ax2[1].plot(JEseries,np.mean(variance_full_theo[:,:,1],axis=1),color='blue',linewidth=1.5,label=r'$\Delta_F^I$',alpha=0.5)
ax2[0].plot(JEseries,np.mean(variance_R1_theo[:,:,0],axis=1),color='orange',linewidth=1.5,label=r'$\Delta_{R1}^E$',alpha=0.5)
ax2[1].plot(JEseries,np.mean(variance_R1_theo[:,:,1],axis=1),color='green',linewidth=1.5,label=r'$\Delta_{R1}^I$',alpha=0.5)

xticks = np.linspace(.0,0.8,2)
xlims = [-0.1,0.9]
yticks = np.linspace(.0,0.3,2)
ylims = [-.05,0.35]

for i in range(2):     
        # ax2[i].set_xlim(xlims)
        # ax2[i].set_ylim(ylims)
        # ax2[i].set_xticks(xticks)
        # ax2[i].set_yticks(yticks)
        ax2[i].legend()

NameError: name 'JEseries' is not defined

## with Symmetric Random Connections -- Rank one unperturbed matrix

In [ ]:
'''
WITH SYMMETRY, 
* increase eta
* constant and homogeneous g0 # randomness
'''
from functools import partial
# generate mean matrix
nrank=1
Nt=np.array([300,300])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
nrank,ntrial,neta,nvec,nmaxFP=1,30,5,2,3
etaseries = np.linspace(0.0,1.0,neta)
''' Network Setting  '''
## heterogeneous random gain
gaverage = 0.5
xee,xei,xie,xii=1.0,1.0,1.0,1.0 

## heterogeneous degree of symmetry: amplitudes and signs
coeffetaEsub=np.array([1.0,1.0,1.0])#
coeffetaTotal = np.array([1.0,1.0,1.0])#
# coeffetaTotal = np.zeros(3)
signetaEsub=np.ones(3)
signetaTotal=np.ones(3)
# signetaTotal[2]=-1

ppercentEsub = np.ones(2)
ppercentEsub[0]=0.5
ppercentEsub[1]=1.0-ppercentEsub[0]
## E->total ppercentEsub[0]/2.0,ppercentEsub[1]/2.0, (I) 1/2.0

'''  Recording Variables. '''
## perturbed + symmetric eigenvectors
Reigvecseries,Leigvecseries=np.zeros((neta,ntrial,NE*2,nvec*2),dtype=complex),np.zeros((neta,ntrial,NE*2,nvec*2),dtype=complex)
ReigvecTseries,LeigvecTseries=np.zeros((neta,ntrial,NE*2,nvec*2),dtype=complex),np.zeros((neta,ntrial,NE*2,nvec*2),dtype=complex)
Beigvseries=np.zeros((neta,ntrial,N),dtype=complex)
## corresponding statistics \mu and \sigma \cov
armu,sigrcov = np.zeros((neta,ntrial,2),dtype=complex),np.zeros((neta,ntrial,2),dtype=complex) # 2 for E and I
almu,siglcov = np.zeros((neta,ntrial,2),dtype=complex),np.zeros((neta,ntrial,2),dtype=complex)
siglr = np.zeros((neta,ntrial,2),dtype=complex)
## reference, eigenvectors of matrix with iid perturbations
x0series,y0series=np.zeros((neta,ntrial,NE*2),dtype=complex),np.zeros((neta,ntrial,NE*2),dtype=complex)
gYbasexee=np.zeros((ntrial,NE*2,NE*2),dtype=complex)
x0series,y0series=np.zeros((ntrial,NE*2),dtype=complex),np.zeros((ntrial,NE*2),dtype=complex)
''' Nullclines parameters  '''
kappaintersect_Full = np.zeros((neta,ntrial,nmaxFP*2))
kappaintersect_R1   = np.zeros((neta,ntrial,nmaxFP*2))

''' simulating parameters '''
tt = np.linspace(0,150,500)
dt = tt[2]-tt[1]
ntt = len(tt)
xfpseries_Full = np.zeros((neta,ntrial,NE*2,ntt))
xfpseries_R1   = np.zeros((neta,ntrial,NE*2,ntt))
''' ## Three \bar{J} cases '''
JI,JE,a,b=0.5,1.4,0,0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
''' Am -- J(g0=0), xAm(g0=0), yAm(g0=0) '''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
''' for reference, eigenvectors of unperturbed matrix \bar{J}'''
meigvecAm,neigvecAm,eigvAm = decompunperturbedEI(Am)
xAm,yAm=np.reshape(meigvecAm[:,0],(NE*2,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(NE*2,1))
''' Iterative Processing '''
for iktrial in range(ntrial):
    print('>>>>>>> simulating neuronal activity ... >>>>>>')
    '''    ##>>>>>>>>>>> g0!=0, >>>>>>>>     '''
    Xsym =iidGaussian([0,gaverage/np.sqrt(NE*2)],[NE*2,NE*2])
    XsymT=Xsym.copy().T
    X0=Xsym.copy()
    ## log: there is a bug, X is the total random xconn, including symmetry
    gYbasexee[iktrial,:,:]=Xsym.copy()

    ## REFERENCE -- J(\rho=0,g=g_0)
    J0=Am.copy()+X0.copy()
    ''' # calculate the original \bar{m} and \bar{n} '''
    eigvJ0,leig0,reig0,x0,y0=decompNormalization(J0,xAm,yAm,sort=0)
    # eigvJ0,eigrvecJ0=la.eig(J0)
    # inveigrvecJ0=la.inv(eigrvecJ0)
    # meig0 = np.squeeze(eigrvecJ0[:,:].copy())
    # neig0 = np.squeeze(inveigrvecJ0[:,:].copy()) # inverse
    # neig0=neig0.copy().T
    # for i in range(nrank):
    #     neig0[:,i]*=eigvJ0[i]
    # for j in range(nrank,N):
    #     neig0[:,j]*=eigvJ0[j]
    # leig0, reig0 = np.zeros((N,N)),np.zeros((N,N))
    # ''' The first '''
    # reig0  =meig0.copy()
    # normval=np.sum(reig0*neig0.copy(),axis=0)
    # normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
    # leig0=neig0.copy()/normval.copy()
    # ## for reference
    # tildex,tildey=np.reshape(reig0[:,0].copy(),(NE*2,1)),np.reshape(leig0[:,0].copy(),(NE*2,1))
    # ## make sure the E in y is positive, for negative change signs of x and y
    # if (np.mean(tildey[:NE,0])*yAm[0,0])<0:
    #     tildex*=(-1)
    #     tildey*=(-1)
    # x0,y0=np.reshape(tildex,(NE*2,1)),np.reshape(tildey,(NE*2,1))
    # x0=np.sqrt(np.squeeze(tildey.T@xAm)/np.squeeze(yAm.T@tildex))*tildex
    # y0=np.sqrt(np.squeeze(tildex.T@yAm)/np.squeeze(xAm.T@tildey))*tildey
    # record references
    x0series[iktrial,:],y0series[iktrial,:]=x0[:,0].copy(),y0[:,0].copy()
    for idxeta,eta in enumerate(etaseries):
        # etaset=np.ones(6)
        # etaset[5]*=eta
        etaset=eta*np.ones(6)
        for iloop in range(3):
            etaset[iloop]   = etaset[iloop]*coeffetaEsub[iloop]
            etaset[iloop+3] = etaset[iloop+3]*coeffetaTotal[iloop]
        Xinit = Xsym.copy() 

        ### >>>>>>>>.Subcircuit Exc sym >>>>>>>>>.
        ## remaining
        hNE=int(ppercentEsub[0]*NE) 
        asqr=(1-np.sqrt(1-etaset[0]**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[:hNE,:hNE]=signetaEsub[0]*aamp*XsymT[:hNE,:hNE].copy()+np.sqrt(1-aamp**2)*Xsym[:hNE,:hNE].copy()
        ## >>>>>>. esub middle
        asqr=(1-np.sqrt(1-etaset[1]**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[:hNE,hNE:NE]=signetaEsub[1]*aamp*XsymT[:hNE,hNE:NE].copy()+np.sqrt(1-aamp**2)*Xsym[:hNE,hNE:NE].copy()
        Xinit[hNE:NE,:hNE]=signetaEsub[1]*aamp*XsymT[hNE:NE,:hNE].copy()+np.sqrt(1-aamp**2)*Xsym[hNE:NE,:hNE].copy()
        ## >>>>>> E2
        asqr=(1-np.sqrt(1-etaset[2]**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[hNE:NE,hNE:NE]=signetaEsub[2]*aamp*XsymT[hNE:NE,hNE:NE].copy()+np.sqrt(1-aamp**2)*Xsym[hNE:NE,hNE:NE].copy()
        ### >>>>>>> Total >>>>>
        # E1I-B
        hNE = int(ppercentEsub[0]*NE) 
        asqr=(1-np.sqrt(1-(etaset[3])**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[NE:,:hNE]=signetaTotal[0]*aamp*XsymT[NE:,:hNE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,:hNE].copy()
        Xinit[:hNE,NE:]=signetaTotal[0]*aamp*XsymT[:hNE,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[:hNE,NE:].copy()
        # E2I-B
        asqr=(1-np.sqrt(1-(etaset[4])**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[NE:,hNE:NE]=signetaTotal[1]*aamp*XsymT[NE:,hNE:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,hNE:NE].copy()
        Xinit[hNE:NE,NE:]=signetaTotal[1]*aamp*XsymT[hNE:NE,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[hNE:NE,NE:].copy()
        ## II ##
        asqr=(1-np.sqrt(1-etaset[5]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,NE:]=signetaTotal[2]*aamp*XsymT[NE:,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,NE:].copy()

        X=Xinit.copy()
        X[:NE,:NE]*=(xee)
        X[:NE,NE:]*=(xei)
        X[NE:,:NE]*=(xie)
        X[NE:,NE:]*=(xii)

        # overall
        J = X.copy()+Am.copy()
        JT = J.copy().T
        ''' iid Gaussian randomness '''
        eigvJ,leigvec,reigvec,xnorm0,ynorm0=decompNormalization(J,x0,y0,sort=1)
        # eigvJ,leigvec,reigvec,xnorm0,ynorm0=decompNormalization(J,xAm,yAm,sort=1)
        # eigvJ,leigvec,reigvec,xnorm0,ynorm0=decompUnnormalization(J,xAm,yAm,sort=1)
        ''' ## P = mn.T/N, already have rl.T -- r(m) \lambda l.T(n.T) -- Reig*\sqrt(N)Leig*\sqrt(N)/N   '''
        xnorm0,ynorm0 = xnorm0*np.sqrt(2*NE),ynorm0*np.sqrt(2*NE)
        Reigvecseries[idxeta,iktrial,:,0],Leigvecseries[idxeta,iktrial,:,0]=xnorm0[:,0].copy(),ynorm0[:,0].copy()
        Beigvseries[idxeta,iktrial,:]=eigvJ.copy()

        axrmu,aylmu,sigxr,sigyl,sigcov = numerical_stats(xnorm0,ynorm0,x0*np.sqrt(NE*2),y0*np.sqrt(NE*2),eigvJ,nrank,2,ppercent=np.array([0.5,0.5]))
        armu[idxeta,iktrial,:],almu[idxeta,iktrial,:] = axrmu[:,0],aylmu[:,0]
        sigrcov[idxeta,iktrial,:],siglcov[idxeta,iktrial,:]= sigxr[:,0],sigyl[:,0]
        siglr[idxeta,iktrial,:] = sigcov[:,0,0]
        
        ## Rank One Approximation, Nullclines of \kappa
        xnorm0, ynorm0 = Reigvecseries[idxeta,iktrial,:,0].copy(),Leigvecseries[idxeta,iktrial,:,0]
        xnorm0, ynorm0 = np.reshape(xnorm0,(2*NE,1)),np.reshape(ynorm0,(2*NE,1))
        xmu,ymu = armu[idxeta,iktrial,:].copy(),almu[idxeta,iktrial,:].copy()
        xsig,ysig = sigrcov[idxeta,iktrial,:].copy(),siglcov[idxeta,iktrial,:].copy()
        yxcov = siglr[idxeta,iktrial,:].copy()

        ''' Full Connectivity -- Dynamics '''
        Jpt = J.copy()
        xinit = np.random.normal(0,1E-2,(1,NE*2))
        xinit = np.squeeze(xinit)
        xtemporal = odesimulation(tt,xinit,Jpt,0)
        xfpseries_Full[idxeta,iktrial,:,:] = xtemporal.T.copy()
        ## 2 KAPPA_0
        kappanum = np.zeros(3)
        xact = np.squeeze(xfpseries_Full[idxeta,iktrial,:,-1])
        ## use yAm -- unperturbed centre.
        if((ymu[0]*eigvJ[0])*(yAm[0,0]*eigvAm[0])>0):
            kappanum[0] = yAm[0,0]*np.mean(np.tanh(xact[:NE]))/2.0+yAm[NE,0]*np.mean(np.tanh(xact[NE:]))/2.0
        else:
            kappanum[0] = -yAm[0,0]*np.mean(np.tanh(xact[:NE]))/2.0-yAm[NE,0]*np.mean(np.tanh(xact[NE:]))/2.0
        kappanum[0]=kappanum[0]*np.sqrt(NE*2)*eigvAm[0]
        kappaintersect_Full[idxeta,iktrial,:3]=kappanum[:].copy()

        ''' use perturbation theorem to calculate the perturbed vectors ''' 
        # xiid,yiid = Xsym.copy()@xAm.copy()/eigvAm[0], Xsym.copy().T@yAm.copy()/eigvAm[0]
        # xiid,yiid = xiid+xAm.copy(),yiid+yAm.copy()
        # xnormt, ynormt = (X.copy()-Xsym.copy())@xiid.copy()/eigvAm[0], (X.copy()-Xsym.copy()).T@yiid.copy()/eigvAm[0]
        # xnormt, ynormt = xnormt+xiid.copy(), ynormt+yiid.copy()
        # xnormt,ynormt  = xnormt*np.sqrt(2*NE),ynormt*np.sqrt(2*NE)
        # ReigvecTseries[idxeta,iktrial,:,0],LeigvecTseries[idxeta,iktrial,:,0]=xnormt[:,0].copy(),ynormt[:,0].copy()

        xnormt, ynormt = (X.copy()-Xsym.copy())@x0.copy()/eigvJ0[0], (X.copy()-Xsym.copy()).T@y0.copy()/eigvJ0[0]
        xnormt, ynormt = xnormt+x0.copy(), ynormt+y0.copy()
        ## normalize 
        xnormt = xnormt.copy()/np.linalg.norm(xnormt.copy())
        ynormt = ynormt.copy()
        _,_,xnormt,ynormt=Normalization(xnormt.copy(),ynormt.copy(),x0.copy(),y0.copy(),sort=0,nrank=1)
        xnormt,ynormt = xnormt*np.sqrt(NE*2),ynormt*np.sqrt(NE*2)
        ReigvecTseries[idxeta,iktrial,:,0],LeigvecTseries[idxeta,iktrial,:,0]=xnormt[:,0].copy(),ynormt[:,0].copy()

        # xnormt,ynormt  = xnorm0,ynorm0
        r1Matt = np.real(xnormt@ynormt.T)
        r1Matt = r1Matt*np.real(eigvJ[0])/NE/2.0
        ### START SIMULATING AND CALCULATE KAPPA AND xfpsereis_R1
        ## use the same initial values
        xtemporal = odesimulation(tt,xinit,r1Matt,0)
        xfpseries_R1[idxeta,iktrial,:,:] = xtemporal.T.copy()
        ''' kappa dynamics    '''
        ## 2 populations 
        kappanum = np.zeros(3)
        xact = np.reshape(np.squeeze(xfpseries_R1[idxeta,iktrial,:,-1]),(NE*2,1))
        kappanum[0] = np.squeeze(ynormt.copy().T@np.tanh(xact.copy()))*eigvJ[0]/NE/2
        kappaintersect_R1[idxeta,iktrial,:3]=kappanum[:].copy()



In [ ]:
fig,ax = plt.subplots(figsize=(4,4))
idxeta,iktrial = 2,10
ax.scatter(Reigvecseries[idxeta,iktrial,:],ReigvecTseries[idxeta,iktrial,:],s=3,alpha=0.5)
ax.plot([0,3],[0,3],c='k')
ax.set_aspect('equal')
fig,ax = plt.subplots(figsize=(4,4))
ax.scatter(Leigvecseries[idxeta,iktrial,:],LeigvecTseries[idxeta,iktrial,:],s=3,alpha=0.5)
ax.plot([0,3],[0,3],c='k')
ax.set_aspect('equal')

In [9]:
## comparing theoretical and numerical kappa, in two networks
## theoretical kappa only exists for R1, there is self-consistency
kappa_theo_iid = np.zeros((neta,3))
for idxeta in range(neta):
    etaset = etaseries[idxeta]*np.ones(6)
    # etaset = np.ones(6)#etaseries[idxeta]*np.ones(6)
    # etaset[5]*=(-etaseries[idxeta])
    init_k=2.0
    kappa_theo_iid[idxeta,0]= fsolve(symperturbation,init_k,args=(JE,JI,gaverage,etaset))
    init_k=0.0
    kappa_theo_iid[idxeta,1]= fsolve(symperturbation,init_k,args=(JE,JI,gaverage,etaset))
    init_k=-2.0
    kappa_theo_iid[idxeta,2]= fsolve(symperturbation,init_k,args=(JE,JI,gaverage,etaset))

## plot kappa change with JE ##
fig,ax = plt.subplots(figsize=(4,4))
ax.plot(etaseries,kappa_theo_iid[:,0],color='black',linewidth=1.5)
ax.plot(etaseries,kappa_theo_iid[:,1],color='gray',linewidth=1.5)
ax.plot(etaseries,kappa_theo_iid[:,2],color='black',linewidth=1.5)
for iktrial in range(ntrial):
    ax.scatter(etaseries,kappaintersect_Full[:,iktrial,0],s=20.0,color='blue',edgecolors='blue',marker='^',alpha=0.5)
    ax.scatter(etaseries,kappaintersect_R1[:,iktrial,0],s=20.0,color='green',edgecolors='green',marker='s',alpha=0.5)


In [10]:
## test two expressions of variance ##
variance_x_num,variance_kappa_num = np.zeros((neta,ntrial,2)),np.zeros((neta,ntrial,2))
mu_x_num,mu_kappa_num = np.zeros((neta,ntrial,2)),np.zeros((neta,ntrial,2))
variance_gavg_num, mu_gavg_num =np.zeros((neta,ntrial,2)), np.zeros((neta,ntrial,2))
for idxje in range(neta):
    for iktrial in range(ntrial):
        ## numerical results for Full Mat
        variance_x_num[idxje,iktrial,0],variance_x_num[idxje,iktrial,1]=np.std(xfpseries_Full[idxje,iktrial,:NE,-1])**2,np.std(xfpseries_Full[idxje,iktrial,NE:,-1])**2
        mu_x_num[idxje,iktrial,0],mu_x_num[idxje,iktrial,1]=np.mean(xfpseries_Full[idxje,iktrial,:NE,-1]),np.mean(xfpseries_Full[idxje,iktrial,NE:,-1])
        
        variance_gavg_num[idxje,iktrial,0],variance_gavg_num[idxje,iktrial,1]= gaverage**2*np.mean(np.tanh(xfpseries_Full[idxje,iktrial,:NE,-1])**2),gaverage**2*np.mean(np.tanh(xfpseries_Full[idxje,iktrial,NE:,-1])**2)
        ## KAPPA_0
        kappa0 = JE*np.mean(np.tanh(xfpseries_Full[idxje,iktrial,:NE,-1]))-JI*np.mean(np.tanh(xfpseries_Full[idxje,iktrial,NE:,-1]))
        mu_gavg_num[idxje,iktrial,0],mu_gavg_num[idxje,iktrial,1]=np.abs(kappa0),np.abs(kappa0)

        ## numerical results for Rank one Appriximation Ma
        variance_kappa_num[idxje,iktrial,0],variance_kappa_num[idxje,iktrial,1]=np.std(xfpseries_R1[idxje,iktrial,:NE,-1])**2,np.std(xfpseries_R1[idxje,iktrial,NE:,-1])**2
        mu_kappa_num[idxje,iktrial,0],mu_kappa_num[idxje,iktrial,1]=np.mean(xfpseries_R1[idxje,iktrial,:NE,-1]),np.mean(xfpseries_R1[idxje,iktrial,NE:,-1])

mu_x_num,mu_kappa_num = np.abs(mu_x_num),np.abs(mu_kappa_num)
xticks = np.linspace(.0,.20,2)
xlims = [0.0,0.2]
yticks = np.linspace(.0,0.20,2)
ylims = [0.0,0.2]
fig,ax2 = plt.subplots(1,2,figsize=(5,4))
xxx=np.array([1,3])
for iktrial in range(ntrial):
        ax2[0].plot(variance_x_num[:,iktrial,0],variance_kappa_num[:,iktrial,0],'^',markersize=1,color='red')
        ax2[1].plot(variance_x_num[:,iktrial,1],variance_kappa_num[:,iktrial,1],'^',markersize=1,color='blue')

xxx = np.linspace(0.0,0.50,2)
for i in range(2):
        ax2[i].plot(xxx,xxx,linestyle='--',color='black') 
        # ax2[i].set_xlim(xlims)
        # ax2[i].set_ylim(ylims)
        # ax2[i].set_xticks(xticks)
        # ax2[i].set_yticks(yticks)
        ax2[i].set_aspect('equal')

xticks = np.linspace(.0,0.8,2)
xlims = [0,0.8]
yticks = np.linspace(.0,.80,2)
ylims = [0,0.8]

fig,ax2 = plt.subplots(1,2,figsize=(5,4))
for iktrial in range(ntrial):
        ax2[0].plot(np.abs(mu_x_num[:,iktrial,0]),np.abs(mu_kappa_num[:,iktrial,0]),'^',markersize=1,color='red')
        ax2[1].plot(np.abs(mu_x_num[:,iktrial,1]),np.abs(mu_kappa_num[:,iktrial,1]),'^',markersize=1,color='blue')

xxx = np.linspace(0.0,1.50,2)
for i in range(2):
        ax2[i].plot(xxx,xxx,linestyle='--',color='black') 
        # ax2[i].set_xlim(xlims)
        # ax2[i].set_ylim(ylims)
        # ax2[i].set_xticks(xticks)
        # ax2[i].set_yticks(yticks)
        ax2[i].set_aspect('equal')


In [ ]:
def add_label(violin, label):
    color = violin["bodies"][0].get_facecolor().flatten()
    labels.append((mpatches.Patch(color=color), label))
labels = []
''' mean mu  '''
binlen = 6
fig,ax2 = plt.subplots(2,1,figsize=(4,4))
positions = np.arange(0, binlen*neta-1, binlen)
add_label(ax2[0].violinplot(
        np.squeeze(np.abs(mu_x_num[:,:,0])).T, positions, showmeans=False, showmedians=True,
        showextrema=False), "num. Full")   
add_label(ax2[1].violinplot(
        np.squeeze(np.abs(mu_x_num[:,:,1])).T, positions, showmeans=False, showmedians=True,
        showextrema=False), "num. Full")    


positions = np.arange(1, binlen*neta-1, binlen)
add_label(ax2[0].violinplot(
        np.squeeze(np.abs(mu_kappa_num[:,:,0])).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"num. Rank-one")
add_label(ax2[1].violinplot(
        np.squeeze(np.abs(mu_kappa_num[:,:,1])).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"num. Rank-one")
positions = np.arange(3, binlen*neta-1, binlen)
add_label(ax2[0].violinplot(
        np.squeeze(mu_gavg_num[:,:,0]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"theo. Full")
add_label(ax2[1].violinplot(
        np.squeeze(mu_gavg_num[:,:,1]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"theo. Full")


### WAY ONE --- for variance
labels = []
fig,ax2 = plt.subplots(2,1,figsize=(4,4))
positions = np.arange(0, binlen*neta-1, binlen)
add_label(ax2[0].violinplot(
        np.squeeze(variance_x_num[:,:,0]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), "num. Full")   
add_label(ax2[1].violinplot(
        np.squeeze(variance_x_num[:,:,1]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), "num. Full")    

positions = np.arange(1, binlen*neta-1, binlen)
add_label(ax2[0].violinplot(
        np.squeeze(variance_kappa_num[:,:,0]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"num. Rank-one")
add_label(ax2[1].violinplot(
        np.squeeze(variance_kappa_num[:,:,1]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"num. Rank-one")

positions = np.arange(3, binlen*neta-1, binlen)
add_label(ax2[0].violinplot(
        np.squeeze(variance_gavg_num[:,:,0]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"theo. Full")
add_label(ax2[1].violinplot(
        np.squeeze(variance_gavg_num[:,:,1]).T, positions, showmeans=False, showmedians=True,
        showextrema=False), r"theo. Full")



* Rank one approximation captures the changes in statistical properties of network with reciprocal connections

In [ ]:
## Figures reflecting how mean and variance change with random gain of iid Gaussian matrix
fig,ax2 = plt.subplots(2,1,figsize=(6,6))
for iktrial in range(ntrial):
    ax2[0].scatter(etaseries-0.02,mu_x_num[:,iktrial,0],s=8.0,color='red',edgecolors='red',marker='^',alpha=0.5)
    ax2[1].scatter(etaseries-0.02,mu_x_num[:,iktrial,1],s=8.0,color='blue',edgecolors='blue',marker='^',alpha=0.5)
    ax2[0].scatter(etaseries+0.02,mu_kappa_num[:,iktrial,0],s=8.0,color='orange',edgecolors='orange',marker='s',alpha=0.5)
    ax2[1].scatter(etaseries+0.02,mu_kappa_num[:,iktrial,1],s=8.0,color='green',edgecolors='green',marker='s',alpha=0.5)
    ax2[0].scatter(etaseries,mu_gavg_num[:,iktrial,0],s=8.0,color='black',edgecolors='black',marker='o',alpha=0.5)
    ax2[1].scatter(etaseries,mu_gavg_num[:,iktrial,1],s=8.0,color='black',edgecolors='black',marker='o',alpha=0.5)

ax2[0].plot(etaseries,np.mean(mu_x_num[:,:,0],axis=1),color='red',linewidth=1.5,label=r'$\mu_F^E$',alpha=0.5)
ax2[1].plot(etaseries,np.mean(mu_x_num[:,:,1],axis=1),color='blue',linewidth=1.5,label=r'$\mu_F^I$',alpha=0.5)
ax2[0].plot(etaseries,np.mean(mu_kappa_num[:,:,0],axis=1),color='orange',linewidth=1.5,label=r'$\mu_{R1}^E$',alpha=0.5)
ax2[1].plot(etaseries,np.mean(mu_kappa_num[:,:,1],axis=1),color='green',linewidth=1.5,label=r'$\mu_{R1}^I$',alpha=0.5)
ax2[0].plot(etaseries,np.mean(mu_gavg_num[:,:,0],axis=1),color='black',linewidth=1.5,label=r'$\mu_{ignore}^E$',alpha=0.5)
ax2[1].plot(etaseries,np.mean(mu_gavg_num[:,:,1],axis=1),color='black',linewidth=1.5,label=r'$\mu_{ignore}^I$',alpha=0.5)

ax2[0].plot(etaseries,np.mean(mu_x_num[0,:,0])*np.ones_like(etaseries),color='gray',linewidth=1.5,linestyle='--',label=r'$\mu_{iid}^E$',alpha=0.5)
ax2[1].plot(etaseries,np.mean(mu_x_num[0,:,1])*np.ones_like(etaseries),color='gray',linewidth=1.5,linestyle='--',label=r'$\mu_{iid}^I$',alpha=0.5)

ax2[0].legend()
ax2[1].legend()
fig,ax2 = plt.subplots(2,1,figsize=(6,6))
for iktrial in range(ntrial):
    ax2[0].scatter(etaseries-0.02,variance_x_num[:,iktrial,0],s=8.0,color='red',edgecolors='red',marker='^',alpha=0.5)
    ax2[1].scatter(etaseries-0.02,variance_x_num[:,iktrial,1],s=8.0,color='blue',edgecolors='blue',marker='^',alpha=0.5)
    ax2[0].scatter(etaseries+0.02,variance_kappa_num[:,iktrial,0],s=8.0,color='orange',edgecolors='orange',marker='s',alpha=0.5)
    ax2[1].scatter(etaseries+0.02,variance_kappa_num[:,iktrial,1],s=8.0,color='green',edgecolors='green',marker='s',alpha=0.5)
    ax2[0].scatter(etaseries,variance_gavg_num[:,iktrial,0],s=8.0,color='black',edgecolors='black',marker='o',alpha=0.5)
    ax2[1].scatter(etaseries,variance_gavg_num[:,iktrial,1],s=8.0,color='black',edgecolors='black',marker='o',alpha=0.5)

ax2[0].plot(etaseries,np.mean(variance_x_num[:,:,0],axis=1),color='red',linewidth=1.5,label=r'$\Delta_F^E$',alpha=0.5)
ax2[1].plot(etaseries,np.mean(variance_x_num[:,:,1],axis=1),color='blue',linewidth=1.5,label=r'$\Delta_F^I$',alpha=0.5)
ax2[0].plot(etaseries,np.mean(variance_kappa_num[:,:,0],axis=1),color='orange',linewidth=1.5,label=r'$\Delta_{R1}^E$',alpha=0.5)
ax2[1].plot(etaseries,np.mean(variance_kappa_num[:,:,1],axis=1),color='green',linewidth=1.5,label=r'$\Delta_{R1}^I$',alpha=0.5)
ax2[0].plot(etaseries,np.mean(variance_gavg_num[:,:,0],axis=1),color='black',linewidth=1.5,label=r'$\Delta_{ignore}^E$',alpha=0.5)
ax2[1].plot(etaseries,np.mean(variance_gavg_num[:,:,1],axis=1),color='black',linewidth=1.5,label=r'$\Delta_{ignore}^I$',alpha=0.5)

ax2[0].plot(etaseries,np.mean(variance_x_num[0,:,0])*np.ones_like(etaseries),color='gray',linewidth=1.5,linestyle='--',label=r'$\Delta_{iid}^E$',alpha=0.5)
ax2[1].plot(etaseries,np.mean(variance_x_num[0,:,1])*np.ones_like(etaseries),color='gray',linewidth=1.5,linestyle='--',label=r'$\Delta_{iid}^I$',alpha=0.5)

ax2[0].legend()
ax2[1].legend()

# Backup Codes

In [ ]:
idxje,iktrial=2,12
print('RIGHT:',np.std(Reigvecseries[idxje,iktrial,:NE,0].flatten()))
print(np.std(ReigvecTseries[idxje,iktrial,:NE,0].flatten())*np.sqrt(NE*2))
print(np.std(Reigvecseries[idxje,iktrial,NE:,0].flatten()))
print(np.std(ReigvecTseries[idxje,iktrial,NE:,0].flatten())*np.sqrt(NE*2))
print(gaverage/(JE-JI))
print('LEFT:',np.std(Leigvecseries[idxje,iktrial,:NE,0].flatten()))
print(np.std(LeigvecTseries[idxje,iktrial,:NE,0].flatten())*np.sqrt(NE*2))
print(np.std(Leigvecseries[idxje,iktrial,NE:,0].flatten()))
print(np.std(LeigvecTseries[idxje,iktrial,NE:,0].flatten())*np.sqrt(NE*2))

print(np.sqrt(2)*np.sqrt(JE**2+JI**2)*gaverage/(JE-JI)**2)

print('RIGHT:',np.mean(Reigvecseries[idxje,iktrial,:NE,0].flatten()))
print(np.mean(ReigvecTseries[idxje,iktrial,:NE,0].flatten())*np.sqrt(NE*2))
print(np.mean(Reigvecseries[idxje,iktrial,NE:,0].flatten()))
print(np.mean(ReigvecTseries[idxje,iktrial,NE:,0].flatten())*np.sqrt(NE*2))

print('LEFT:',np.mean(Leigvecseries[idxje,iktrial,:NE,0].flatten()))
print(np.mean(LeigvecTseries[idxje,iktrial,:NE,0].flatten())*np.sqrt(NE*2))
print(np.mean(Leigvecseries[idxje,iktrial,NE:,0].flatten()))
print(np.mean(LeigvecTseries[idxje,iktrial,NE:,0].flatten())*np.sqrt(NE*2))

print(2*JE/(JE-JI),-2*JI/(JE-JI))

print(np.sum(Reigvecseries[idxje,iktrial,:,0]**2))
print(np.sum(Reigvecseries[idxje,iktrial,:,0]*Leigvecseries[idxje,iktrial,:,0]))
# print(gaverage/(JE-JI))

In [ ]:
Xsym =iidGaussian([0,gaverage/np.sqrt(NE*4)],[NE*4,NE*4])
XsymT = Xsym.copy().T
u,s,v = la.svd(Xsym)
u_,s_,v_ = la.svd((Xsym+XsymT)/np.sqrt(2))
print(np.shape(s_))
eigv,eigvec=la.eig((Xsym))
eigv_,eigvec_=la.eig((Xsym+XsymT)/np.sqrt(2))
print(gaverage)

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(5,4))
ax[0].hist(s,bins=20)
ax[0].hist(s_,bins=20)
fig,ax = plt.subplots(figsize=(4,4))
ax.scatter(np.real(eigv_),np.imag(eigv_))
fig,ax = plt.subplots(figsize=(4,4))
ax.scatter(np.real(Beigvseries[1,1,:]),np.imag(Beigvseries[1,1,:]))

In [ ]:
print((1+etaseries)*gaverage)